In [2]:
!pip install scikit-posthocs

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re
from scipy.stats import shapiro, levene, f_oneway, kruskal
import scikit_posthocs as sp_post
import statsmodels.api as sm
from statsmodels.formula.api import ols
import os
import tifffile as tiff

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cell_line_map = {'APP/PS1':['PF336', 'PF339', 'PF342'],
                 'APP/PS1-FIRE':['PF346', 'PF369', 'PF415'],
                 'FIRE':['PF305', 'PF384', 'PF389'],
                 'WT':['PF332', 'PF386', 'PF390']}

## Check how many stacks and slices per stack on batch data

In [6]:
a_input_folders = ['/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_input/1. PSD95/1. Original File Names/',
                '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_input/2. SY38/1. Original File Names/',
               '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_input/3. MXO4/1. Original File Names/',
               '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_input/4. GFAP/1. Original File Names/'
               ]


a_gt_folders = [
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_output/1. PSD95/2. Decrypted File Names/',
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_output/2. SY38/2. Decrypted File Names/',
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_output/3. MXO4/2. Decrypted File Names/',
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_output/4. GFAP/2. Decrypted File Names/']

b_input_folders = ['/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_input/1. PSD95/1. Original File Names/',
                '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_input/2. SY38/1. Original File Names/',
               '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_input/3. MXO4/1. Original File Names/',
               '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_input/4. GFAP/1. Original File Names/'
               ]


b_gt_folders = [
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_output/1. PSD95/2. Decrypted File Names/',
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_output/2. SY38/2. Decrypted File Names/',
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_output/3. MXO4/2. Decrypted File Names/',
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_output/4. GFAP/2. Decrypted File Names/']

In [9]:
file_names = sorted(os.listdir(a_input_folders[0]))
file_names = [f for f in file_names if f.endswith('.tif')]

In [75]:
new_list = [s.split('-')[0] for s in file_names]

In [80]:
file_names = sorted(os.listdir(a_input_folders[0]))
file_names = [f for f in file_names if f.endswith('.tif')]

# Create a reversed mapping from mouse ID to cell line
mouse_to_line_map = {}
for line, mice in cell_line_map.items():
    for mouse in mice:
        mouse_to_line_map[mouse] = line

# Count occurrences of each cell line in the extracted parts
cell_line_counts = {}
for part in file_names:
    # The extracted part is the mouse ID (e.g., 'PF390')
    mouse_id = part.split('-')[0] # Assuming the mouse ID is the first part before '-'
    line = mouse_to_line_map.get(mouse_id)
    if line:
        cell_line_counts[line] = cell_line_counts.get(line, 0) + 1
    else:
        print(f"Warning: Mouse ID '{mouse_id}' from '{part}' not found in cell_line_map.")


print("Number of elements mapped to each cell line:")
for line, count in cell_line_counts.items():
    print(f"{line}: {count}")

Number of elements mapped to each cell line:
FIRE: 4
APP/PS1: 4
APP/PS1-FIRE: 4
WT: 4


In [82]:
file_names = sorted(os.listdir(b_input_folders[0]))
file_names = [f for f in file_names if f.endswith('.tif')]

# Create a reversed mapping from mouse ID to cell line
mouse_to_line_map = {}
for line, mice in cell_line_map.items():
    for mouse in mice:
        mouse_to_line_map[mouse] = line

# Count occurrences of each cell line in the extracted parts
cell_line_counts = {}
for part in file_names:
    # The extracted part is the mouse ID (e.g., 'PF390')
    mouse_id = part.split('-')[0] # Assuming the mouse ID is the first part before '-'
    line = mouse_to_line_map.get(mouse_id)
    if line:
        cell_line_counts[line] = cell_line_counts.get(line, 0) + 1
    else:
        print(f"Warning: Mouse ID '{mouse_id}' from '{part}' not found in cell_line_map.")


print("Number of elements mapped to each cell line:")
for line, count in cell_line_counts.items():
    print(f"{line}: {count}")

Number of elements mapped to each cell line:
WT: 12
APP/PS1: 12
APP/PS1-FIRE: 12
FIRE: 12


In [83]:
file_names = sorted(os.listdir(a_input_folders[0]))
file_names = [f for f in file_names if f.endswith('.tif')]
image_names = []
for i, file_name in enumerate(file_names):
    input_img = tiff.imread(a_input_folders[0]+file_name)
    image_names.append(input_img)
print(len(image_names))
total_slices = [len(i) for i in image_names]
print(total_slices)
print(sum(total_slices))

16
[26, 24, 26, 25, 37, 16, 29, 19, 25, 27, 16, 26, 17, 11, 20, 13]
357


In [30]:
file_names = sorted(os.listdir(b_input_folders[0]))
file_names = [f for f in file_names if f.endswith('.tif')]
image_names = []
for i, file_name in enumerate(file_names):
    input_img = tiff.imread(b_input_folders[0]+file_name)
    image_names.append(input_img)
print(len(image_names))
total_slices = [len(i) for i in image_names]
print(total_slices)
print(sum(total_slices))

48
[25, 22, 27, 13, 11, 14, 18, 18, 21, 24, 25, 13, 37, 20, 21, 18, 11, 11, 32, 27, 32, 14, 29, 22, 11, 18, 17, 26, 27, 27, 9, 28, 13, 22, 20, 17, 10, 29, 15, 30, 14, 29, 11, 11, 13, 26, 15, 23]
966


In [32]:
966+357

1323

## Calculate number of objects per lab mask

In [33]:
from scipy.ndimage import label

In [65]:
file_names = sorted(os.listdir(b_gt_folders[0]))
file_names = [f for f in file_names if f.endswith('.tif')]
image_names = []
num_masks = []
for i, file_name in enumerate(file_names):
    gt_img = tiff.imread(b_gt_folders[0]+file_name)
    gt_img, num_points = label(gt_img)
    image_names.append(gt_img)
    num_masks.append(num_points)
print(len(image_names))
total_slices = [len(i) for i in image_names]
print(sum(total_slices))

num_objects = []
for i in range(len(image_names)):
  for j in range(len(image_names[i])):
   _, num_masks = label(image_names[i][j])
  num_objects.append(num_masks)

non_zero = [x for x in num_objects if x != 0]
print("Proportion of non-zero objects", len(non_zero)/len(num_objects))

print("Min:", min(non_zero))
print("Median", np.median(non_zero))
print("Max", max(non_zero))
print('\n')

file_names = sorted(os.listdir(a_gt_folders[0]))
file_names = [f for f in file_names if f.endswith('.tif')]
image_names = []
num_masks = []
for i, file_name in enumerate(file_names):
    gt_img = tiff.imread(a_gt_folders[0]+file_name)
    gt_img, num_points = label(gt_img)
    image_names.append(gt_img)
    num_masks.append(num_points)
print(len(image_names))
total_slices = [len(i) for i in image_names]
print(sum(total_slices))

num_objects = []
for i in range(len(image_names)):
  for j in range(len(image_names[i])):
   _, num_masks = label(image_names[i][j])
  num_objects.append(num_masks)

non_zero = [x for x in num_objects if x != 0]
print("Proportion of non-zero objects", len(non_zero)/len(num_objects))

print("Min:", min(non_zero))
print("Median", np.median(non_zero))
print("Max", max(non_zero))
print('\n')

48
966
Proportion of non-zero objects 1.0
Min: 423
Median 939.5
Max 1458


16
357
Proportion of non-zero objects 1.0
Min: 793
Median 1026.5
Max 1511




In [66]:
file_names = sorted(os.listdir(a_gt_folders[1]))
file_names = [f for f in file_names if f.endswith('.tif')]
image_names = []
num_masks = []
for i, file_name in enumerate(file_names):
    gt_img = tiff.imread(a_gt_folders[1]+file_name)
    gt_img, num_points = label(gt_img)
    image_names.append(gt_img)
    num_masks.append(num_points)
print(len(image_names))
total_slices = [len(i) for i in image_names]
print(sum(total_slices))

num_objects = []
for i in range(len(image_names)):
  for j in range(len(image_names[i])):
   _, num_masks = label(image_names[i][j])
  num_objects.append(num_masks)
non_zero = [x for x in num_objects if x != 0]
print("Proportion of non-zero objects", len(non_zero)/len(num_objects))

print("Min:", min(non_zero))
print("Median", np.median(non_zero))
print("Max", max(non_zero))
print('\n')

file_names = sorted(os.listdir(b_gt_folders[1]))
file_names = [f for f in file_names if f.endswith('.tif')]
image_names = []
num_masks = []
for i, file_name in enumerate(file_names):
    gt_img = tiff.imread(b_gt_folders[1]+file_name)
    gt_img, num_points = label(gt_img)
    image_names.append(gt_img)
    num_masks.append(num_points)
print(len(image_names))
total_slices = [len(i) for i in image_names]
print(sum(total_slices))

num_objects = []
for i in range(len(image_names)):
  for j in range(len(image_names[i])):
   _, num_masks = label(image_names[i][j])
  num_objects.append(num_masks)
non_zero = [x for x in num_objects if x != 0]
print("Proportion of non-zero objects", len(non_zero)/len(num_objects))

print("Min:", min(non_zero))
print("Median", np.median(non_zero))
print("Max", max(non_zero))
print('\n')

16
357
Proportion of non-zero objects 1.0
Min: 1115
Median 1895.5
Max 2224


48
966
Proportion of non-zero objects 1.0
Min: 1012
Median 1598.5
Max 2310




In [63]:
file_names = sorted(os.listdir(a_gt_folders[2]))
file_names = [f for f in file_names if f.endswith('.tif')]
image_names = []
num_masks = []
for i, file_name in enumerate(file_names):
    gt_img = tiff.imread(a_gt_folders[2]+file_name)
    gt_img, num_points = label(gt_img)
    image_names.append(gt_img)
    num_masks.append(num_points)
print(len(image_names))
total_slices = [len(i) for i in image_names]
print(sum(total_slices))

num_objects = []
for i in range(len(image_names)):
  for j in range(len(image_names[2])):
    try:
      _, num_masks = label(image_names[i][j])
    except:
      num_masks = 0
  num_objects.append(num_masks)


non_zero = [x for x in num_objects if x != 0]
print("Proportion of non-zero objects", len(non_zero)/len(num_objects))

print("Min:", min(non_zero))
print("Median", np.median(non_zero))
print("Max", max(non_zero))
print('\n')



file_names = sorted(os.listdir(b_gt_folders[2]))
file_names = [f for f in file_names if f.endswith('.tif')]
image_names = []
num_masks = []
for i, file_name in enumerate(file_names):
    gt_img = tiff.imread(b_gt_folders[2]+file_name)
    gt_img, num_points = label(gt_img)
    image_names.append(gt_img)
    num_masks.append(num_points)
print(len(image_names))
total_slices = [len(i) for i in image_names]
print(sum(total_slices))

num_objects = []
for i in range(len(image_names)):
  for j in range(len(image_names[i])):
   _, num_masks = label(image_names[i][j])
  num_objects.append(num_masks)

non_zero = [x for x in num_objects if x != 0]
print("Proportion of non-zero objects", len(non_zero)/len(num_objects))

print("Min:", min(non_zero))
print("Median", np.median(non_zero))
print("Max", max(non_zero))
print('\n')

16
357
Proportion of non-zero objects 0.1875
Min: 2
Median 39.0
Max 47


48
966
Proportion of non-zero objects 0.5625
Min: 1
Median 5.0
Max 167




In [64]:
file_names = sorted(os.listdir(a_gt_folders[3]))
file_names = [f for f in file_names if f.endswith('.tif')]
image_names = []
num_masks = []
for i, file_name in enumerate(file_names):
    gt_img = tiff.imread(a_gt_folders[3]+file_name)
    gt_img, num_points = label(gt_img)
    image_names.append(gt_img)
    num_masks.append(num_points)
print(len(image_names))
total_slices = [len(i) for i in image_names]
print(sum(total_slices))

num_objects = []
for i in range(len(image_names)):
  for j in range(len(image_names[i])):
   _, num_masks = label(image_names[i][j])
  num_objects.append(num_masks)

non_zero = [x for x in num_objects if x != 0]
print("Proportion of non-zero objects", len(non_zero)/len(num_objects))

print("Min:", min(non_zero))
print("Median", np.median(non_zero))
print("Max", max(non_zero))
print('\n')

file_names = sorted(os.listdir(b_gt_folders[3]))
file_names = [f for f in file_names if f.endswith('.tif')]
image_names = []
num_masks = []
for i, file_name in enumerate(file_names):
    gt_img = tiff.imread(b_gt_folders[3]+file_name)
    gt_img, num_points = label(gt_img)
    image_names.append(gt_img)
    num_masks.append(num_points)
print(len(image_names))
total_slices = [len(i) for i in image_names]
print(sum(total_slices))

num_objects = []
for i in range(len(image_names)):
  for j in range(len(image_names[i])):
   _, num_masks = label(image_names[i][j])
  num_objects.append(num_masks)

non_zero = [x for x in num_objects if x != 0]
print("Proportion of non-zero objects", len(non_zero)/len(num_objects))

print("Min:", min(non_zero))
print("Median", np.median(non_zero))
print("Max", max(non_zero))
print('\n')

16
369
Proportion of non-zero objects 0.375
Min: 3
Median 15.5
Max 35


48
966
Proportion of non-zero objects 0.5
Min: 1
Median 11.5
Max 76




## Results files

#### SAM2

In [83]:
concat_df = pd.DataFrame(columns = ['Stain', 'Run_mode','File_name', 'Dice','IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct'])

file_list = ['train_test_same_batch_2001_tune_results.csv', 'train_test_diff_batch_2001_tune_results.csv', 'train_test_mixed_batch_2001_tune_results.csv']
run_mode_list = ['same_batch', 'diff_batch', 'mixed_batch']
for c, run_mode in zip(file_list, run_mode_list):
  df_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/results/Results_files/SAM2/SY38/' + c)
  df_file = df_file[['Dice Coefficient', 'IoU (Jaccard Index)', 'Precision', 'Recall', 'Model Pixels Label %', 'Lab Pixels Label %', 'File name']]
  df_file.columns = ['Dice', 'IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct', 'File_name']
  df_file['Run_mode'] = run_mode
  df_file['Stain'] = 'SY38'
  df_file = df_file[['Stain', 'Run_mode', 'File_name', 'Dice','IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct']]
  concat_df = pd.concat([concat_df, df_file])



file_list = ['train_test_same_batch_2001_tune_results.csv', 'train_test_diff_batch_2001_tune_results.csv', 'train_test_mixed_batch_2001_tune_results.csv']
run_mode_list = ['same_batch', 'diff_batch', 'mixed_batch']
for c, run_mode in zip(file_list, run_mode_list):
  df_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/results/Results_files/SAM2/PSD/' + c)
  df_file = df_file[['Dice Coefficient', 'IoU (Jaccard Index)', 'Precision', 'Recall', 'Model Pixels Label %', 'Lab Pixels Label %', 'File name']]
  df_file.columns = ['Dice', 'IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct', 'File_name']
  df_file['Run_mode'] = run_mode
  df_file['Stain'] = 'PSD'
  df_file = df_file[['Stain', 'Run_mode', 'File_name', 'Dice','IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct']]
  concat_df = pd.concat([concat_df, df_file])



file_list = ['train_test_same_batch_2000_tune_results.csv', 'train_test_diff_batch_2000_tune_results.csv', 'train_test_mixed_batch_2000_tune_results.csv']
run_mode_list = ['same_batch', 'diff_batch', 'mixed_batch']
for c, run_mode in zip(file_list, run_mode_list):
  df_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/results/Results_files/SAM2/MXO4/' + c)
  df_file = df_file[['Dice Coefficient', 'IoU (Jaccard Index)', 'Precision', 'Recall', 'Model Pixels Label %', 'Lab Pixels Label %', 'File name']]
  df_file.columns = ['Dice', 'IoU',  'Precision', 'Recall','Model_pct', 'Lab_pct', 'File_name']
  df_file['Run_mode'] = run_mode
  df_file['Stain'] = 'MXO4'
  df_file = df_file[['Stain', 'Run_mode', 'File_name', 'Dice','IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct']]
  concat_df = pd.concat([concat_df, df_file])


file_list = ['train_test_same_batch_2001_tune_results.csv', 'train_test_diff_batch_2001_tune_results.csv', 'train_test_mixed_batch_2001_tune_results.csv']
run_mode_list = ['same_batch', 'diff_batch', 'mixed_batch']
for c, run_mode in zip(file_list, run_mode_list):
  df_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/results/Results_files/SAM2/GFAP/' + c)
  df_file = df_file[['Dice Coefficient', 'IoU (Jaccard Index)', 'Precision', 'Recall', 'Model Pixels Label %', 'Lab Pixels Label %', 'File name']]
  df_file.columns = ['Dice', 'IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct', 'File_name']
  df_file['Run_mode'] = run_mode
  df_file['Stain'] = 'GFAP'
  df_file = df_file[['Stain', 'Run_mode', 'File_name', 'Dice','IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct']]
  concat_df = pd.concat([concat_df, df_file])


concat_df.columns = ['stain', 'run_mode','file_name', 'dice','iou', 'precision', 'recall', 'model_pct', 'lab_pct']
concat_df['mouse'] = concat_df['file_name'].str.split('-').str[0]
# Create a reversed mapping from mouse ID to cell line
mouse_to_line_map = {}
for line, mice in cell_line_map.items():
    for mouse in mice:
        mouse_to_line_map[mouse] = line

# Apply the new mapping to create the 'Line' column
concat_df['line'] = concat_df['mouse'].map(mouse_to_line_map)
sam_df = concat_df.copy()
sam_df['file'] = sam_df['file_name'].str.split('.').str[0]

/tmp/ipython-input-2713127983.py:12: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [84]:
sam_df[sam_df['lab_pct'] > 0.001].groupby(['stain', 'run_mode']).agg({"dice":"median", "precision":"median", "recall":"median"}).sort_values(by=['stain', 'run_mode'])

dice  precision    recall
stain run_mode                                  
GFAP  diff_batch   0.689163   0.968942  0.540675
      mixed_batch  0.765812   0.829672  0.761992
      same_batch   0.753869   0.794777  0.748372
MXO4  diff_batch   0.855497   0.789082  0.962663
      mixed_batch  0.865234   0.974921  0.796556
      same_batch   0.860062   0.792672  0.981786
PSD   diff_batch   0.527517   0.593572  0.493794
      mixed_batch  0.564656   0.656543  0.502007
      same_batch   0.521724   0.634255  0.450067
SY38  diff_batch   0.506425   0.701903  0.393339
      mixed_batch  0.600317   0.722033  0.518901
      same_batch   0.560395   0.695033  0.498623

In [45]:
concat_df = pd.DataFrame(columns = ['Stain', 'Run_mode','File_name', 'Dice','IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct'])

file_list = ['train_test_same_batch_2001_tune_summarised_results.csv', 'train_test_diff_batch_2001_tune_summarised_results.csv', 'train_test_mixed_batch_2001_tune_summarised_results.csv']
run_mode_list = ['same_batch', 'diff_batch', 'mixed_batch']
for c, run_mode in zip(file_list, run_mode_list):
  df_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/results/Results_files/SAM2/SY38/' + c)
  df_file = df_file[['Dice Coefficient', 'IoU (Jaccard Index)', 'Precision', 'Recall', 'Model Pixels Label %', 'Lab Pixels Label %', 'File name']]
  df_file.columns = ['Dice', 'IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct', 'File_name']
  df_file['Run_mode'] = run_mode
  df_file['Stain'] = 'SY38'
  df_file = df_file[['Stain', 'Run_mode', 'File_name', 'Dice','IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct']]
  concat_df = pd.concat([concat_df, df_file])



file_list = ['train_test_same_batch_2001_tune_summarised_results.csv', 'train_test_diff_batch_2001_tune_summarised_results.csv', 'train_test_mixed_batch_2001_tune_summarised_results.csv']
run_mode_list = ['same_batch', 'diff_batch', 'mixed_batch']
for c, run_mode in zip(file_list, run_mode_list):
  df_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/results/Results_files/SAM2/PSD/' + c)
  df_file = df_file[['Dice Coefficient', 'IoU (Jaccard Index)', 'Precision', 'Recall', 'Model Pixels Label %', 'Lab Pixels Label %', 'File name']]
  df_file.columns = ['Dice', 'IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct', 'File_name']
  df_file['Run_mode'] = run_mode
  df_file['Stain'] = 'PSD'
  df_file = df_file[['Stain', 'Run_mode', 'File_name', 'Dice','IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct']]
  concat_df = pd.concat([concat_df, df_file])



file_list = ['train_test_same_batch_2000_tune_summarised_results.csv', 'train_test_diff_batch_2000_tune_summarised_results.csv', 'train_test_mixed_batch_2000_tune_summarised_results.csv']
run_mode_list = ['same_batch', 'diff_batch', 'mixed_batch']
for c, run_mode in zip(file_list, run_mode_list):
  df_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/results/Results_files/SAM2/MXO4/' + c)
  df_file = df_file[['Dice Coefficient', 'IoU (Jaccard Index)', 'Precision', 'Recall', 'Model Pixels Label %', 'Lab Pixels Label %', 'File name']]
  df_file.columns = ['Dice', 'IoU',  'Precision', 'Recall','Model_pct', 'Lab_pct', 'File_name']
  df_file['Run_mode'] = run_mode
  df_file['Stain'] = 'MXO4'
  df_file = df_file[['Stain', 'Run_mode', 'File_name', 'Dice','IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct']]
  concat_df = pd.concat([concat_df, df_file])


file_list = ['train_test_same_batch_2001_tune_summarised_results.csv', 'train_test_diff_batch_2001_tune_summarised_results.csv', 'train_test_mixed_batch_2001_tune_summarised_results.csv']
run_mode_list = ['same_batch', 'diff_batch', 'mixed_batch']
for c, run_mode in zip(file_list, run_mode_list):
  df_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/results/Results_files/SAM2/GFAP/' + c)
  df_file = df_file[['Dice Coefficient', 'IoU (Jaccard Index)', 'Precision', 'Recall', 'Model Pixels Label %', 'Lab Pixels Label %', 'File name']]
  df_file.columns = ['Dice', 'IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct', 'File_name']
  df_file['Run_mode'] = run_mode
  df_file['Stain'] = 'GFAP'
  df_file = df_file[['Stain', 'Run_mode', 'File_name', 'Dice','IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct']]
  concat_df = pd.concat([concat_df, df_file])


concat_df.columns = ['stain', 'run_mode','file_name', 'dice','iou', 'precision', 'recall', 'model_pct', 'lab_pct']
concat_df['mouse'] = concat_df['file_name'].str.split('-').str[0]
# Create a reversed mapping from mouse ID to cell line
mouse_to_line_map = {}
for line, mice in cell_line_map.items():
    for mouse in mice:
        mouse_to_line_map[mouse] = line

# Apply the new mapping to create the 'Line' column
concat_df['line'] = concat_df['mouse'].map(mouse_to_line_map)
sam_sum_df = concat_df.copy()


/tmp/ipython-input-2006751167.py:12: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [46]:
sam_sum_df

,stain,run_mode,file_name,dice,iou,precision,recall,model_pct,lab_pct,mouse,line
0,SY38,same_batch,PF390-S2-Stack-A-CROPPED-sy38,0.590076,0.418516,0.807674,0.464841,0.092356,0.160471,PF390,WT
1,SY38,same_batch,PF390-S1-Stack-C-CROPPED-sy38,0.347705,0.210438,0.742513,0.227003,0.024393,0.079787,PF390,WT
2,SY38,same_batch,PF415-S2-Stack-C-CROPPED-sy38,0.588174,0.416605,0.649132,0.537681,0.087171,0.105239,PF415,APP/PS1-FIRE
3,SY38,same_batch,PF390-S2-Stack-B-CROPPED-sy38,0.372194,0.228648,0.751044,0.247398,0.021081,0.063996,PF390,WT
4,SY38,same_batch,PF390-S2-Stack-C-CROPPED-sy38,0.524682,0.355640,0.807644,0.388551,0.080580,0.167494,PF390,WT
...,...,...,...,...,...,...,...,...,...,...,...
11,GFAP,mixed_batch,PF390-S2-Stack-C-CROPPED-gfap,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PF390,WT
12,GFAP,mixed_batch,PF390-S2-Stack-A-CROPPED-gfap,0.663569,0.496523,0.514409,0.934555,0.000887,0.000488,PF390,WT
13,GFAP,mixed_batch,PF386-S2-Stack-A-CROPPED-gfap,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PF386,WT
14,GFAP,mixed_batch,PF390-S1-Stack-B-CROPPED-gfap,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PF390,WT


### Cellpose

In [71]:
concat_df = pd.DataFrame(columns = ['Stain', 'Run_mode','File_name', 'Dice','IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct'])


file_list =  ['train_test_same_batch_501_tune_results.csv', 'train_test_diff_batch_501_tune_results.csv', 'train_test_mixed_batch_501_tune_results.csv']
run_mode_list = ['same_batch', 'diff_batch', 'mixed_batch']
for c, run_mode in zip(file_list, run_mode_list):
  df_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/results/Results_files/Cellpose/SY38/' + c)
  df_file = df_file[['Dice Coefficient', 'IoU (Jaccard Index)', 'Precision', 'Recall', 'Model Pixels Label %', 'Lab Pixels Label %', 'File name']]
  df_file.columns = ['Dice', 'IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct', 'File_name']
  df_file['Run_mode'] = run_mode
  df_file['Stain'] = 'SY38'
  df_file = df_file[['Stain', 'Run_mode', 'File_name', 'Dice','IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct']]
  concat_df = pd.concat([concat_df, df_file])



file_list =  ['train_test_same_batch_501_tune_results.csv', 'train_test_diff_batch_501_tune_results.csv', 'train_test_mixed_batch_501_tune_results.csv']
run_mode_list = ['same_batch', 'diff_batch', 'mixed_batch']
for c, run_mode in zip(file_list, run_mode_list):
  df_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/results/Results_files/Cellpose/PSD/' + c)
  df_file = df_file[['Dice Coefficient', 'IoU (Jaccard Index)', 'Precision', 'Recall', 'Model Pixels Label %', 'Lab Pixels Label %', 'File name']]
  df_file.columns = ['Dice', 'IoU',  'Precision', 'Recall','Model_pct', 'Lab_pct', 'File_name']
  df_file['Run_mode'] = run_mode
  df_file['Stain'] = 'PSD'
  df_file = df_file[['Stain', 'Run_mode', 'File_name', 'Dice','IoU',  'Precision', 'Recall','Model_pct', 'Lab_pct']]
  concat_df = pd.concat([concat_df, df_file])



file_list =  ['train_test_same_batch_101_tune_results.csv', 'train_test_diff_batch_101_tune_results.csv', 'train_test_mixed_batch_101_tune_results.csv']
run_mode_list = ['same_batch', 'diff_batch', 'mixed_batch']
for c, run_mode in zip(file_list, run_mode_list):
  df_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/results/Results_files/Cellpose/MXO4/' + c)
  df_file = df_file[['Dice Coefficient', 'IoU (Jaccard Index)', 'Precision', 'Recall', 'Model Pixels Label %', 'Lab Pixels Label %', 'File name']]
  df_file.columns = ['Dice', 'IoU',  'Precision', 'Recall','Model_pct', 'Lab_pct', 'File_name']
  df_file['Run_mode'] = run_mode
  df_file['Stain'] = 'MXO4'
  df_file = df_file[['Stain', 'Run_mode', 'File_name', 'Dice','IoU',  'Precision', 'Recall','Model_pct', 'Lab_pct']]
  concat_df = pd.concat([concat_df, df_file])


file_list =  ['train_test_same_batch_101_tune_results.csv', 'train_test_diff_batch_101_tune_results.csv', 'train_test_mixed_batch_101_tune_results.csv']
run_mode_list = ['same_batch', 'diff_batch', 'mixed_batch']
for c, run_mode in zip(file_list, run_mode_list):
  df_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/results/Results_files/Cellpose/GFAP/' + c)
  df_file = df_file[['Dice Coefficient', 'IoU (Jaccard Index)', 'Precision', 'Recall', 'Model Pixels Label %', 'Lab Pixels Label %', 'File name']]
  df_file.columns = ['Dice', 'IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct', 'File_name']
  df_file['Run_mode'] = run_mode
  df_file['Stain'] = 'GFAP'
  df_file = df_file[['Stain', 'Run_mode', 'File_name', 'Dice','IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct']]
  concat_df = pd.concat([concat_df, df_file])



concat_df.columns = ['stain', 'run_mode','file_name', 'dice','iou',  'precision', 'recall', 'model_pct', 'lab_pct']
concat_df['mouse'] = concat_df['file_name'].str.split('-').str[0]
# Create a reversed mapping from mouse ID to cell line
mouse_to_line_map = {}
for line, mice in cell_line_map.items():
    for mouse in mice:
        mouse_to_line_map[mouse] = line

# Apply the new mapping to create the 'Line' column
concat_df['line'] = concat_df['mouse'].map(mouse_to_line_map)
cell_df = concat_df.copy()
cell_df['file'] = cell_df['file_name'].str.split('.').str[0]

/tmp/ipython-input-3287326225.py:13: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [72]:
cell_df

,stain,run_mode,file_name,dice,iou,precision,recall,model_pct,lab_pct,mouse,line,file
0,SY38,same_batch,PF390-S1-Stack-B-CROPPED-sy38.tif_image_22,0.562891,0.391683,0.402866,0.933821,0.113675,0.049041,PF390,WT,PF390-S1-Stack-B-CROPPED-sy38
1,SY38,same_batch,PF390-S1-Stack-B-CROPPED-sy38.tif_image_23,0.572993,0.401535,0.413784,0.931340,0.114604,0.050917,PF390,WT,PF390-S1-Stack-B-CROPPED-sy38
2,SY38,same_batch,PF390-S1-Stack-B-CROPPED-sy38.tif_image_24,0.575660,0.404159,0.414628,0.941201,0.114607,0.050488,PF390,WT,PF390-S1-Stack-B-CROPPED-sy38
3,SY38,same_batch,PF390-S1-Stack-B-CROPPED-sy38.tif_image_25,0.552216,0.381422,0.394057,0.922454,0.112193,0.047927,PF390,WT,PF390-S1-Stack-B-CROPPED-sy38
4,SY38,same_batch,PF390-S1-Stack-B-CROPPED-sy38.tif_image_26,0.567950,0.396599,0.408589,0.931108,0.115299,0.050596,PF390,WT,PF390-S1-Stack-B-CROPPED-sy38
...,...,...,...,...,...,...,...,...,...,...,...,...
263,GFAP,mixed_batch,PF386-S2-Stack-B-CROPPED-gfap.tif_image_8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PF386,WT,PF386-S2-Stack-B-CROPPED-gfap
264,GFAP,mixed_batch,PF386-S2-Stack-B-CROPPED-gfap.tif_image_9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PF386,WT,PF386-S2-Stack-B-CROPPED-gfap
265,GFAP,mixed_batch,PF386-S2-Stack-B-CROPPED-gfap.tif_image_10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PF386,WT,PF386-S2-Stack-B-CROPPED-gfap
266,GFAP,mixed_batch,PF386-S2-Stack-B-CROPPED-gfap.tif_image_11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PF386,WT,PF386-S2-Stack-B-CROPPED-gfap


In [81]:
cell_df[cell_df['lab_pct'] > 0.001].groupby(['stain', 'run_mode']).agg({"dice":"median", "precision":"median", "recall":"median"}).sort_values(by=['stain', 'run_mode'])

dice  precision    recall
stain run_mode                                  
GFAP  diff_batch   0.845049   0.926119  0.809228
      mixed_batch  0.778119   0.951348  0.649087
      same_batch   0.827147   0.939763  0.742918
MXO4  diff_batch   0.836431   0.769986  0.946577
      mixed_batch  0.846122   0.933839  0.834916
      same_batch   0.903603   0.890009  0.938264
PSD   diff_batch   0.543064   0.391127  0.886060
      mixed_batch  0.598651   0.461742  0.864248
      same_batch   0.628725   0.491128  0.876767
SY38  diff_batch   0.533835   0.381834  0.890455
      mixed_batch  0.540999   0.395546  0.855581
      same_batch   0.537699   0.377949  0.934816

In [50]:
concat_df = pd.DataFrame(columns = ['Stain', 'Run_mode','File_name', 'Dice','IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct'])


file_list =  ['train_test_same_batch_501_tune_summarised_results.csv', 'train_test_diff_batch_501_tune_summarised_results.csv', 'train_test_mixed_batch_501_tune_summarised_results.csv']
run_mode_list = ['same_batch', 'diff_batch', 'mixed_batch']
for c, run_mode in zip(file_list, run_mode_list):
  df_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/results/Results_files/Cellpose/SY38/' + c)
  df_file = df_file[['Dice Coefficient', 'IoU (Jaccard Index)', 'Precision', 'Recall', 'Model Pixels Label %', 'Lab Pixels Label %', 'File name']]
  df_file.columns = ['Dice', 'IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct', 'File_name']
  df_file['Run_mode'] = run_mode
  df_file['Stain'] = 'SY38'
  df_file = df_file[['Stain', 'Run_mode', 'File_name', 'Dice','IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct']]
  concat_df = pd.concat([concat_df, df_file])



file_list =  ['train_test_same_batch_501_tune_summarised_results.csv', 'train_test_diff_batch_501_tune_summarised_results.csv', 'train_test_mixed_batch_501_tune_summarised_results.csv']
run_mode_list = ['same_batch', 'diff_batch', 'mixed_batch']
for c, run_mode in zip(file_list, run_mode_list):
  df_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/results/Results_files/Cellpose/PSD/' + c)
  df_file = df_file[['Dice Coefficient', 'IoU (Jaccard Index)', 'Precision', 'Recall', 'Model Pixels Label %', 'Lab Pixels Label %', 'File name']]
  df_file.columns = ['Dice', 'IoU',  'Precision', 'Recall','Model_pct', 'Lab_pct', 'File_name']
  df_file['Run_mode'] = run_mode
  df_file['Stain'] = 'PSD'
  df_file = df_file[['Stain', 'Run_mode', 'File_name', 'Dice','IoU',  'Precision', 'Recall','Model_pct', 'Lab_pct']]
  concat_df = pd.concat([concat_df, df_file])



file_list =  ['train_test_same_batch_101_tune_summarised_results.csv', 'train_test_diff_batch_101_tune_summarised_results.csv', 'train_test_mixed_batch_101_tune_summarised_results.csv']
run_mode_list = ['same_batch', 'diff_batch', 'mixed_batch']
for c, run_mode in zip(file_list, run_mode_list):
  df_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/results/Results_files/Cellpose/MXO4/' + c)
  df_file = df_file[['Dice Coefficient', 'IoU (Jaccard Index)', 'Precision', 'Recall', 'Model Pixels Label %', 'Lab Pixels Label %', 'File name']]
  df_file.columns = ['Dice', 'IoU',  'Precision', 'Recall','Model_pct', 'Lab_pct', 'File_name']
  df_file['Run_mode'] = run_mode
  df_file['Stain'] = 'MXO4'
  df_file = df_file[['Stain', 'Run_mode', 'File_name', 'Dice','IoU',  'Precision', 'Recall','Model_pct', 'Lab_pct']]
  concat_df = pd.concat([concat_df, df_file])


file_list =  ['train_test_same_batch_101_tune_summarised_results.csv', 'train_test_diff_batch_101_tune_summarised_results.csv', 'train_test_mixed_batch_101_tune_summarised_results.csv']
run_mode_list = ['same_batch', 'diff_batch', 'mixed_batch']
for c, run_mode in zip(file_list, run_mode_list):
  df_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/results/Results_files/Cellpose/GFAP/' + c)
  df_file = df_file[['Dice Coefficient', 'IoU (Jaccard Index)', 'Precision', 'Recall', 'Model Pixels Label %', 'Lab Pixels Label %', 'File name']]
  df_file.columns = ['Dice', 'IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct', 'File_name']
  df_file['Run_mode'] = run_mode
  df_file['Stain'] = 'GFAP'
  df_file = df_file[['Stain', 'Run_mode', 'File_name', 'Dice','IoU', 'Precision', 'Recall', 'Model_pct', 'Lab_pct']]
  concat_df = pd.concat([concat_df, df_file])



concat_df.columns = ['stain', 'run_mode','file_name', 'dice','iou',  'precision', 'recall', 'model_pct', 'lab_pct']
concat_df['mouse'] = concat_df['file_name'].str.split('-').str[0]
# Create a reversed mapping from mouse ID to cell line
mouse_to_line_map = {}
for line, mice in cell_line_map.items():
    for mouse in mice:
        mouse_to_line_map[mouse] = line

# Apply the new mapping to create the 'Line' column
concat_df['line'] = concat_df['mouse'].map(mouse_to_line_map)
cell_sum_df = concat_df.copy()

/tmp/ipython-input-1637865344.py:13: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [52]:
cell_df.sort_values(by = ['file_name', 'stain',  'run_mode', ]).head(10)

,stain,run_mode,file_name,dice,iou,precision,recall,model_pct,lab_pct,mouse,line,file
0,GFAP,diff_batch,PF305-S1-Stack-A-CROPPED-gfap.tif_image_0,0.0,0.0,0.0,0.0,0.0,0.0,PF305,FIRE,PF305-S1-Stack-A-CROPPED-gfap
1,GFAP,diff_batch,PF305-S1-Stack-A-CROPPED-gfap.tif_image_1,0.0,0.0,0.0,0.0,0.0,0.0,PF305,FIRE,PF305-S1-Stack-A-CROPPED-gfap
10,GFAP,diff_batch,PF305-S1-Stack-A-CROPPED-gfap.tif_image_10,0.0,0.0,0.0,0.0,0.0,0.0,PF305,FIRE,PF305-S1-Stack-A-CROPPED-gfap
11,GFAP,diff_batch,PF305-S1-Stack-A-CROPPED-gfap.tif_image_11,0.0,0.0,0.0,0.0,0.0,0.0,PF305,FIRE,PF305-S1-Stack-A-CROPPED-gfap
12,GFAP,diff_batch,PF305-S1-Stack-A-CROPPED-gfap.tif_image_12,0.0,0.0,0.0,0.0,0.0,0.0,PF305,FIRE,PF305-S1-Stack-A-CROPPED-gfap
13,GFAP,diff_batch,PF305-S1-Stack-A-CROPPED-gfap.tif_image_13,0.0,0.0,0.0,0.0,0.0,0.0,PF305,FIRE,PF305-S1-Stack-A-CROPPED-gfap
14,GFAP,diff_batch,PF305-S1-Stack-A-CROPPED-gfap.tif_image_14,0.0,0.0,0.0,0.0,0.0,0.0,PF305,FIRE,PF305-S1-Stack-A-CROPPED-gfap
15,GFAP,diff_batch,PF305-S1-Stack-A-CROPPED-gfap.tif_image_15,0.0,0.0,0.0,0.0,0.0,0.0,PF305,FIRE,PF305-S1-Stack-A-CROPPED-gfap
16,GFAP,diff_batch,PF305-S1-Stack-A-CROPPED-gfap.tif_image_16,0.0,0.0,0.0,0.0,0.0,0.0,PF305,FIRE,PF305-S1-Stack-A-CROPPED-gfap
17,GFAP,diff_batch,PF305-S1-Stack-A-CROPPED-gfap.tif_image_17,0.0,0.0,0.0,0.0,0.0,0.0,PF305,FIRE,PF305-S1-Stack-A-CROPPED-gfap


## Batch runs

In [82]:
# Define the order of run_modes to be consistent
df = sam_df.copy()
df = df[df['lab_pct'] > 0.001]
run_modes = df['run_mode'].unique().tolist()
run_modes.sort()

# Create the boxplot (grouped by stain, colored by run_mode)
fig = px.box(
    df,
    x='stain',
    y='precision',
    color='run_mode',
    points='all',
    category_orders={'run_mode': run_modes},
    color_discrete_sequence=px.colors.qualitative.Set2,
)

# Calculate medians per stain and run_mode
medians = df.groupby(['stain', 'run_mode'])['precision'].median().reset_index()

# Number of run_modes (hues)
n_run_modes = len(run_modes)

# For each median, calculate approximate x position
# Plotly groups categories on x-axis as integers 0,1,... for each stain
# Boxes within group are spaced evenly around the integer
# We use a spacing factor (adjust if needed)

# Spacing factor for offset from the integer x position
offset = 0.2

# Create a mapping from stain to its integer position
stain_order = df['stain'].unique().tolist()
stain_pos = {stain: i for i, stain in enumerate(stain_order)}

for _, row in medians.iterrows():
    stain_idx = stain_pos[row['stain']]
    run_mode_idx = run_modes.index(row['run_mode'])

    # Calculate x position offset: centered around stain_idx
    # Spread run_modes evenly around stain_idx
    x_pos = stain_idx - offset*(n_run_modes-1)/2 + run_mode_idx*offset

    fig.add_annotation(
        x=x_pos,
        y=row['precision'],
        text=f"{row['precision']:.3f}",
        showarrow=False,
        yshift=15,
        font=dict(size=12, color='black', family='Arial Black, Arial, sans-serif'),  # bold font family
        align='center',
        xanchor='center',
    )

# Update layout for clarity
fig.update_layout(
    title="SAM2: Precision by Stain and Batch Training Mode",
    yaxis_title="Precision Score",
    xaxis_title="Stain",
    boxmode='group',
    font=dict(size=14),
    legend_title_text="Batch Training Mode",
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(l=50, r=50, t=60, b=50)
)

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()

In [54]:
# Define the order of run_modes to be consistent
df = sam_df.copy()
df = df[df['lab_pct'] > 0.001]
run_modes = df['run_mode'].unique().tolist()
run_modes.sort()

# Create the boxplot (grouped by stain, colored by run_mode)
fig = px.box(
    df,
    x='stain',
    y='recall',
    color='run_mode',
    points='all',
    category_orders={'run_mode': run_modes},
    color_discrete_sequence=px.colors.qualitative.Set2,
)

# Calculate medians per stain and run_mode
medians = df.groupby(['stain', 'run_mode'])['recall'].median().reset_index()

# Number of run_modes (hues)
n_run_modes = len(run_modes)

# For each median, calculate approximate x position
# Plotly groups categories on x-axis as integers 0,1,... for each stain
# Boxes within group are spaced evenly around the integer
# We use a spacing factor (adjust if needed)

# Spacing factor for offset from the integer x position
offset = 0.2

# Create a mapping from stain to its integer position
stain_order = df['stain'].unique().tolist()
stain_pos = {stain: i for i, stain in enumerate(stain_order)}

for _, row in medians.iterrows():
    stain_idx = stain_pos[row['stain']]
    run_mode_idx = run_modes.index(row['run_mode'])

    # Calculate x position offset: centered around stain_idx
    # Spread run_modes evenly around stain_idx
    x_pos = stain_idx - offset*(n_run_modes-1)/2 + run_mode_idx*offset

    fig.add_annotation(
        x=x_pos,
        y=row['recall'],
        text=f"{row['recall']:.3f}",
        showarrow=False,
        yshift=15,
        font=dict(size=12, color='black', family='Arial Black, Arial, sans-serif'),  # bold font family
        align='center',
        xanchor='center',
    )

# Update layout for clarity
fig.update_layout(
    title="SAM2: Recall by Stain and Batch Training Mode",
    yaxis_title="Recall Score",
    xaxis_title="Stain",
    boxmode='group',
    font=dict(size=14),
    legend_title_text="Batch Training Mode",
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(l=50, r=50, t=60, b=50)
)

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()

In [55]:
# Define the order of run_modes to be consistent
df = sam_df.copy()
df = df[df['lab_pct'] > 0.001]
run_modes = df['run_mode'].unique().tolist()
run_modes.sort()

# Create the boxplot (grouped by stain, colored by run_mode)
fig = px.box(
    df,
    x='stain',
    y='dice',
    color='run_mode',
    points='all',
    category_orders={'run_mode': run_modes},
    color_discrete_sequence=px.colors.qualitative.Set2,
)

# Calculate medians per stain and run_mode
medians = df.groupby(['stain', 'run_mode'])['dice'].median().reset_index()

# Number of run_modes (hues)
n_run_modes = len(run_modes)

# For each median, calculate approximate x position
# Plotly groups categories on x-axis as integers 0,1,... for each stain
# Boxes within group are spaced evenly around the integer
# We use a spacing factor (adjust if needed)

# Spacing factor for offset from the integer x position
offset = 0.2

# Create a mapping from stain to its integer position
stain_order = df['stain'].unique().tolist()
stain_pos = {stain: i for i, stain in enumerate(stain_order)}

for _, row in medians.iterrows():
    stain_idx = stain_pos[row['stain']]
    run_mode_idx = run_modes.index(row['run_mode'])

    # Calculate x position offset: centered around stain_idx
    # Spread run_modes evenly around stain_idx
    x_pos = stain_idx - offset*(n_run_modes-1)/2 + run_mode_idx*offset

    fig.add_annotation(
        x=x_pos,
        y=row['dice'],
        text=f"{row['dice']:.3f}",
        showarrow=False,
        yshift=15,
        font=dict(size=12, color='black', family='Arial Black, Arial, sans-serif'),  # bold font family
        align='center',
        xanchor='center',
    )

# Update layout for clarity
fig.update_layout(
    title="SAM2: Dice by Stain and Batch Training Mode",
    yaxis_title="Dice Score",
    xaxis_title="Stain",
    boxmode='group',
    font=dict(size=14),
    legend_title_text="Batch Training Mode",
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(l=50, r=50, t=60, b=50)
)

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()

### Check for significance

In [56]:
results = {}

df = sam_df
for stain in df['stain'].unique():
    subset = df[df['stain'] == stain]
    groups = [subset[subset['run_mode'] == rm]['dice'] for rm in subset['run_mode'].unique()]

    # Assumption checks
    shapiro_p = max(shapiro(subset['dice'])[1] for rm in subset['run_mode'].unique())
    levene_p = levene(*groups)[1]
    print(shapiro_p)
    print(levene_p)

    if shapiro_p > 0.05 and levene_p > 0.05:
        # One-way ANOVA
        model = ols('dice ~ C(run_mode)', data=subset).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        posthoc = sm.stats.multicomp.MultiComparison(subset['dice'], subset['run_mode']).tukeyhsd()
        results[stain] = {"ANOVA": anova_table, "Posthoc": posthoc.summary()}
    else:
        # Kruskal–Wallis + Dunn
        kw_p = kruskal(*groups)[1]
        posthoc = sp_post.posthoc_dunn(subset, val_col='dice', group_col='run_mode', p_adjust='bonferroni')
        results[stain] = {"KW_p": kw_p, "Posthoc": posthoc}

print(results)



summary_rows = []

for stain in df['stain'].unique():
    subset = df[df['stain'] == stain]
    groups = [subset[subset['run_mode'] == rm]['dice'] for rm in subset['run_mode'].unique()]

    # Overall Kruskal–Wallis
    kw_stat, kw_p = kruskal(*groups)

    # Medians per run mode
    medians = subset.groupby('run_mode')['dice'].median()

    # Dunn's post-hoc with Bonferroni adjustment
    posthoc = sp_post.posthoc_dunn(subset, val_col='dice', group_col='run_mode', p_adjust='bonferroni')

    # Extract significant comparisons
    sig_pairs = []
    for i in posthoc.index:
        for j in posthoc.columns:
            if i < j:  # avoid duplicates
                pval = posthoc.loc[i, j]
                if pval < 0.05:
                    sig_pairs.append(f"{i} vs {j} (p={pval:.3g})")

    summary_rows.append({
        "Stain": stain,
        "KW_p": kw_p,
        "Median diff_batch": medians.get('diff_batch', float('nan')),
        "Median mixed_batch": medians.get('mixed_batch', float('nan')),
        "Median same_batch": medians.get('same_batch', float('nan')),
        "Significant comparisons": "; ".join(sig_pairs) if sig_pairs else "None"
    })

summary_df = pd.DataFrame(summary_rows)
print(summary_df)

2.414743369936097e-23
2.958388384740359e-11
1.465517481913628e-27
0.02736198751857534
8.306286743959378e-34
0.6167311970817158
5.380994958408314e-35
0.002511893369469574
{'SY38': {'KW_p': np.float64(9.160221427081595e-14), 'Posthoc':                diff_batch   mixed_batch  same_batch
diff_batch   1.000000e+00  2.787205e-14    0.007030
mixed_batch  2.787205e-14  1.000000e+00    0.000484
same_batch   7.030436e-03  4.842585e-04    1.000000}, 'PSD': {'KW_p': np.float64(4.2909408168068687e-14), 'Posthoc':                diff_batch   mixed_batch    same_batch
diff_batch   1.000000e+00  7.823407e-12  1.000000e+00
mixed_batch  7.823407e-12  1.000000e+00  1.966155e-10
same_batch   1.000000e+00  1.966155e-10  1.000000e+00}, 'MXO4': {'KW_p': np.float64(0.016278829485393425), 'Posthoc':              diff_batch  mixed_batch  same_batch
diff_batch     1.000000     1.000000    0.013841
mixed_batch    1.000000     1.000000    0.107978
same_batch     0.013841     0.107978    1.000000}, 'GFAP': {'KW_p'

In [57]:
summary_df

,Stain,KW_p,Median diff_batch,Median mixed_batch,Median same_batch,Significant comparisons
0,SY38,9.160221e-14,0.506425,0.600317,0.560395,diff_batch vs mixed_batch (p=2.79e-14); diff_b...
1,PSD,4.290941e-14,0.527517,0.564656,0.521724,diff_batch vs mixed_batch (p=7.82e-12); mixed_...
2,MXO4,1.627883e-02,0.000000,0.000000,0.429219,diff_batch vs same_batch (p=0.0138)
3,GFAP,2.617607e-03,0.000000,0.000000,0.000000,diff_batch vs mixed_batch (p=0.00196)


In [58]:
# Define the order of run_modes to be consistent
df = cell_df.copy()
df = df[df['lab_pct'] > 0.001]
run_modes = df['run_mode'].unique().tolist()
run_modes.sort()

# Create the boxplot (grouped by stain, colored by run_mode)
fig = px.box(
    df,
    x='stain',
    y='precision',
    color='run_mode',
    points='all',
    category_orders={'run_mode': run_modes},
    color_discrete_sequence=px.colors.qualitative.Set2,
)

# Calculate medians per stain and run_mode
medians = df.groupby(['stain', 'run_mode'])['precision'].median().reset_index()

# Number of run_modes (hues)
n_run_modes = len(run_modes)

# For each median, calculate approximate x position
# Plotly groups categories on x-axis as integers 0,1,... for each stain
# Boxes within group are spaced evenly around the integer
# We use a spacing factor (adjust if needed)

# Spacing factor for offset from the integer x position
offset = 0.2

# Create a mapping from stain to its integer position
stain_order = df['stain'].unique().tolist()
stain_pos = {stain: i for i, stain in enumerate(stain_order)}

for _, row in medians.iterrows():
    stain_idx = stain_pos[row['stain']]
    run_mode_idx = run_modes.index(row['run_mode'])

    # Calculate x position offset: centered around stain_idx
    # Spread run_modes evenly around stain_idx
    x_pos = stain_idx - offset*(n_run_modes-1)/2 + run_mode_idx*offset

    fig.add_annotation(
        x=x_pos,
        y=row['precision'],
        text=f"{row['precision']:.3f}",
        showarrow=False,
        yshift=15,
        font=dict(size=12, color='black', family='Arial Black, Arial, sans-serif'),  # bold font family
        align='center',
        xanchor='center',
    )

# Update layout for clarity
fig.update_layout(
    title="Cellpose: Precision by Stain and Batch Training Mode",
    yaxis_title="Precision Score",
    xaxis_title="Stain",
    boxmode='group',
    font=dict(size=14),
    legend_title_text="Batch Training Mode",
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(l=50, r=50, t=60, b=50)
)

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()

In [59]:
# Define the order of run_modes to be consistent
df = cell_df.copy()
df = df[df['lab_pct'] > 0.001]
run_modes = df['run_mode'].unique().tolist()
run_modes.sort()

# Create the boxplot (grouped by stain, colored by run_mode)
fig = px.box(
    df,
    x='stain',
    y='recall',
    color='run_mode',
    points='all',
    category_orders={'run_mode': run_modes},
    color_discrete_sequence=px.colors.qualitative.Set2,
)

# Calculate medians per stain and run_mode
medians = df.groupby(['stain', 'run_mode'])['recall'].median().reset_index()

# Number of run_modes (hues)
n_run_modes = len(run_modes)

# For each median, calculate approximate x position
# Plotly groups categories on x-axis as integers 0,1,... for each stain
# Boxes within group are spaced evenly around the integer
# We use a spacing factor (adjust if needed)

# Spacing factor for offset from the integer x position
offset = 0.2

# Create a mapping from stain to its integer position
stain_order = df['stain'].unique().tolist()
stain_pos = {stain: i for i, stain in enumerate(stain_order)}

for _, row in medians.iterrows():
    stain_idx = stain_pos[row['stain']]
    run_mode_idx = run_modes.index(row['run_mode'])

    # Calculate x position offset: centered around stain_idx
    # Spread run_modes evenly around stain_idx
    x_pos = stain_idx - offset*(n_run_modes-1)/2 + run_mode_idx*offset

    fig.add_annotation(
        x=x_pos,
        y=row['recall'],
        text=f"{row['recall']:.3f}",
        showarrow=False,
        yshift=15,
        font=dict(size=12, color='black', family='Arial Black, Arial, sans-serif'),  # bold font family
        align='center',
        xanchor='center',
    )

# Update layout for clarity
fig.update_layout(
    title="Cellpose: Recall by Stain and Batch Training Mode",
    yaxis_title="Recall Score",
    xaxis_title="Stain",
    boxmode='group',
    font=dict(size=14),
    legend_title_text="Batch Training Mode",
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(l=50, r=50, t=60, b=50)
)

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()

In [60]:
# Define the order of run_modes to be consistent
df = cell_df.copy()
df = df[df['lab_pct'] > 0.001]
run_modes = df['run_mode'].unique().tolist()
run_modes.sort()

# Create the boxplot (grouped by stain, colored by run_mode)
fig = px.box(
    df,
    x='stain',
    y='dice',
    color='run_mode',
    points='all',
    category_orders={'run_mode': run_modes},
    color_discrete_sequence=px.colors.qualitative.Set2,
)

# Calculate medians per stain and run_mode
medians = df.groupby(['stain', 'run_mode'])['dice'].median().reset_index()

# Number of run_modes (hues)
n_run_modes = len(run_modes)

# For each median, calculate approximate x position
# Plotly groups categories on x-axis as integers 0,1,... for each stain
# Boxes within group are spaced evenly around the integer
# We use a spacing factor (adjust if needed)

# Spacing factor for offset from the integer x position
offset = 0.2

# Create a mapping from stain to its integer position
stain_order = df['stain'].unique().tolist()
stain_pos = {stain: i for i, stain in enumerate(stain_order)}

for _, row in medians.iterrows():
    stain_idx = stain_pos[row['stain']]
    run_mode_idx = run_modes.index(row['run_mode'])

    # Calculate x position offset: centered around stain_idx
    # Spread run_modes evenly around stain_idx
    x_pos = stain_idx - offset*(n_run_modes-1)/2 + run_mode_idx*offset

    fig.add_annotation(
        x=x_pos,
        y=row['dice'],
        text=f"{row['dice']:.3f}",
        showarrow=False,
        yshift=15,
        font=dict(size=12, color='black', family='Arial Black, Arial, sans-serif'),  # bold font family
        align='center',
        xanchor='center',
    )

# Update layout for clarity
fig.update_layout(
    title="Cellpose: Dice by Stain and Batch Training Mode",
    yaxis_title="Dice Score",
    xaxis_title="Stain",
    boxmode='group',
    font=dict(size=14),
    legend_title_text="Batch Training Mode",
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(l=50, r=50, t=60, b=50)
)

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()

In [61]:
results = {}

df = cell_df
for stain in df['stain'].unique():
    subset = df[df['stain'] == stain]
    groups = [subset[subset['run_mode'] == rm]['dice'] for rm in subset['run_mode'].unique()]

    # Assumption checks
    shapiro_p = max(shapiro(subset['dice'])[1] for rm in subset['run_mode'].unique())
    levene_p = levene(*groups)[1]
    print(shapiro_p)
    print(levene_p)

    if shapiro_p > 0.05 and levene_p > 0.05:
        # One-way ANOVA
        model = ols('dice ~ C(run_mode)', data=subset).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        posthoc = sm.stats.multicomp.MultiComparison(subset['dice'], subset['run_mode']).tukeyhsd()
        results[stain] = {"ANOVA": anova_table, "Posthoc": posthoc.summary()}
    else:
        # Kruskal–Wallis + Dunn
        kw_p = kruskal(*groups)[1]
        posthoc = sp_post.posthoc_dunn(subset, val_col='dice', group_col='run_mode', p_adjust='bonferroni')
        results[stain] = {"KW_p": kw_p, "Posthoc": posthoc}

print(results)



summary_rows = []

for stain in df['stain'].unique():
    subset = df[df['stain'] == stain]
    groups = [subset[subset['run_mode'] == rm]['dice'] for rm in subset['run_mode'].unique()]

    # Overall Kruskal–Wallis
    kw_stat, kw_p = kruskal(*groups)

    # Medians per run mode
    medians = subset.groupby('run_mode')['dice'].median()

    # Dunn's post-hoc with Bonferroni adjustment
    posthoc = sp_post.posthoc_dunn(subset, val_col='dice', group_col='run_mode', p_adjust='bonferroni')

    # Extract significant comparisons
    sig_pairs = []
    for i in posthoc.index:
        for j in posthoc.columns:
            if i < j:  # avoid duplicates
                pval = posthoc.loc[i, j]
                if pval < 0.05:
                    sig_pairs.append(f"{i} vs {j} (p={pval:.3g})")

    summary_rows.append({
        "Stain": stain,
        "KW_p": kw_p,
        "Median diff_batch": medians.get('diff_batch', float('nan')),
        "Median mixed_batch": medians.get('mixed_batch', float('nan')),
        "Median same_batch": medians.get('same_batch', float('nan')),
        "Significant comparisons": "; ".join(sig_pairs) if sig_pairs else "None"
    })

summary_df = pd.DataFrame(summary_rows)
print(summary_df)

5.014544040860237e-13
5.233051711877136e-14
3.881101151534709e-10
6.8308544552427165e-18
1.85463906745958e-40
0.0013918715313364185
1.2637436537306777e-36
6.472323072292066e-05
{'SY38': {'KW_p': np.float64(0.006033315336920732), 'Posthoc':              diff_batch  mixed_batch  same_batch
diff_batch     1.000000     0.004579    0.319596
mixed_batch    0.004579     1.000000    0.696943
same_batch     0.319596     0.696943    1.000000}, 'PSD': {'KW_p': np.float64(7.685405043687005e-49), 'Posthoc':                diff_batch   mixed_batch    same_batch
diff_batch   1.000000e+00  7.912883e-30  2.178114e-39
mixed_batch  7.912883e-30  1.000000e+00  2.279060e-02
same_batch   2.178114e-39  2.279060e-02  1.000000e+00}, 'MXO4': {'KW_p': np.float64(0.0013835474287422346), 'Posthoc':              diff_batch  mixed_batch  same_batch
diff_batch     1.000000     0.001074    1.000000
mixed_batch    0.001074     1.000000    0.060429
same_batch     1.000000     0.060429    1.000000}, 'GFAP': {'KW_p': np.f

In [62]:
summary_df

,Stain,KW_p,Median diff_batch,Median mixed_batch,Median same_batch,Significant comparisons
0,SY38,6.033315e-03,0.533835,0.540999,0.537699,diff_batch vs mixed_batch (p=0.00458)
1,PSD,7.685405e-49,0.543064,0.598651,0.628725,diff_batch vs mixed_batch (p=7.91e-30); diff_b...
2,MXO4,1.383547e-03,0.000000,0.000000,0.000000,diff_batch vs mixed_batch (p=0.00107)
3,GFAP,1.808249e-05,0.000000,0.000000,0.000000,diff_batch vs mixed_batch (p=9.81e-06); mixed_...


## Pixel Labels by Cell Lines



In [69]:
from scipy.stats import pearsonr, wilcoxon
import plotly.express as px

df = cell_df
for stain in df['stain'].unique():
    df_stain = df[(df['stain'] == stain) & (df['lab_pct'] > 0)].copy() # Filter for lab_pct > 0
    df_stain = df_stain.sort_values(by = 'line')

    if df_stain.empty:
        print(f"No data with lab_pct > 0 for stain: {stain}")
        continue

    min_val = min(df_stain["model_pct"].quantile(0.025),
                  df_stain["lab_pct"].quantile(0.025))
    max_val = max(df_stain["model_pct"].quantile(0.975),
                  df_stain["lab_pct"].quantile(0.975))

    fig = px.scatter(
        df_stain,
        x="model_pct",
        y="lab_pct",
        color="line",
        marginal_x="histogram",  # Histogram along X
        marginal_y="histogram",  # Histogram along Y
        opacity=0.6,
        template="plotly_white"
    )

    # Add 45° reference line
    fig.add_shape(
          type="line",
          x0=min_val,
          y0=min_val,
          x1=max_val,
          y1=max_val,
          line=dict(color="lightgray", width=1, dash="dash")  # lighter and thinner
    )

    # Get correlation and Wilcoxon test results per line and add annotations
    lines = df_stain['line'].unique()
    annotations = []
    for line in lines:
        df_line = df_stain[df_stain['line'] == line].copy() # Create a copy to avoid SettingWithCopyWarning

        if len(df_line) > 1:  # Need at least 2 points for correlation and Wilcoxon
            # Pearson correlation
            try:
                corr, _ = pearsonr(df_line['model_pct'], df_line['lab_pct'])
                corr_text = f"r={corr:.3f}"
            except ValueError:
                 corr_text = "r=NaN"


            # Wilcoxon signed-rank test
            try:
                wilcoxon_stat, wilcoxon_p = wilcoxon(df_line['lab_pct'], df_line['model_pct'])
                wilcoxon_text = f"Wilcoxon p={wilcoxon_p:.3g}"
            except (ValueError, TypeError) as e:
                 wilcoxon_text = f"Wilcoxon test failed: {e}"

            print(f"{line}\n{corr_text}\n{wilcoxon_text}")




    # fig.update_layout(
    #     title=f"{stain}: Model vs Lab Percentages with Distributions (lab_pct > 0)",
    #     xaxis=dict(title="Model (%)", range=[min_val, max_val]),
    #     yaxis=dict(title="Lab (%)", range=[min_val, max_val]),
    #     xaxis_title="Model (%)",
    #     yaxis_title="Lab (%)"
    # )


    # fig.show()

    fig.update_layout(
        title=f"{stain}: Model vs Lab Percentages with Distributions (lab_pct > 0)",
        title_font_size=22,
        font=dict(
            size=16,          # Global font size
            family="Arial"    # Optional: set a font family
        ),
        xaxis=dict(
            title="Model (%)",
            title_font=dict(size=18),
            tickfont=dict(size=14),
            range=[min_val, max_val]
        ),
        yaxis=dict(
            title="Lab (%)",
            title_font=dict(size=18),
            tickfont=dict(size=14),
            range=[min_val, max_val]
        ),
        legend=dict(
            font=dict(size=14)
        ),
      width=1000,   # width in pixels
      height=600   # height in pixels

    )
    fig.show()

APP/PS1
r=0.578
Wilcoxon p=2.67e-18
APP/PS1-FIRE
r=0.090
Wilcoxon p=1.97e-51
FIRE
r=0.082
Wilcoxon p=2.67e-18
WT
r=0.099
Wilcoxon p=6.63e-53


APP/PS1
r=0.350
Wilcoxon p=2.67e-18
APP/PS1-FIRE
r=0.420
Wilcoxon p=1.97e-51
FIRE
r=0.582
Wilcoxon p=2.67e-18
WT
r=0.212
Wilcoxon p=6.63e-53


APP/PS1
r=0.980
Wilcoxon p=0.0296
APP/PS1-FIRE
r=0.896
Wilcoxon p=0.772
FIRE
r=nan
Wilcoxon p=0.0625
WT
r=0.634
Wilcoxon p=6.57e-18


/tmp/ipython-input-2868962508.py:48: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.



APP/PS1
r=0.986
Wilcoxon p=3.36e-09
APP/PS1-FIRE
r=0.848
Wilcoxon p=8.98e-22
FIRE
r=0.864
Wilcoxon p=0.0625
WT
r=0.854
Wilcoxon p=0.000117


In [70]:
from scipy.stats import pearsonr, wilcoxon
import plotly.express as px

df = sam_df
for stain in df['stain'].unique():
    df_stain = df[(df['stain'] == stain) & (df['lab_pct'] > 0)].copy() # Filter for lab_pct > 0
    df_stain = df_stain.sort_values(by = 'line')

    if df_stain.empty:
        print(f"No data with lab_pct > 0 for stain: {stain}")
        continue

    min_val = min(df_stain["model_pct"].quantile(0.025),
                  df_stain["lab_pct"].quantile(0.025))
    max_val = max(df_stain["model_pct"].quantile(0.975),
                  df_stain["lab_pct"].quantile(0.975))

    fig = px.scatter(
        df_stain,
        x="model_pct",
        y="lab_pct",
        color="line",
        marginal_x="histogram",  # Histogram along X
        marginal_y="histogram",  # Histogram along Y
        opacity=0.6,
        template="plotly_white"
    )

    # Add 45° reference line
    fig.add_shape(
          type="line",
          x0=min_val,
          y0=min_val,
          x1=max_val,
          y1=max_val,
          line=dict(color="lightgray", width=1, dash="dash")  # lighter and thinner
    )

    # Get correlation and Wilcoxon test results per line and add annotations
    lines = df_stain['line'].unique()
    annotations = []
    for line in lines:
        df_line = df_stain[df_stain['line'] == line].copy() # Create a copy to avoid SettingWithCopyWarning

        if len(df_line) > 1:  # Need at least 2 points for correlation and Wilcoxon
            # Pearson correlation
            try:
                corr, _ = pearsonr(df_line['model_pct'], df_line['lab_pct'])
                corr_text = f"r={corr:.3f}"
            except ValueError:
                 corr_text = "r=NaN"


            # Wilcoxon signed-rank test
            try:
                wilcoxon_stat, wilcoxon_p = wilcoxon(df_line['lab_pct'], df_line['model_pct'])
                wilcoxon_text = f"Wilcoxon p={wilcoxon_p:.3g}"
            except (ValueError, TypeError) as e:
                 wilcoxon_text = f"Wilcoxon test failed: {e}"

            print(f"{line}\n{corr_text}\n{wilcoxon_text}")




    fig.update_layout(
        title=f"{stain}: Model vs Lab Percentages with Distributions (lab_pct > 0)",
        title_font_size=22,
        font=dict(
            size=16,          # Global font size
            family="Arial"    # Optional: set a font family
        ),
        xaxis=dict(
            title="Model (%)",
            title_font=dict(size=18),
            tickfont=dict(size=14),
            range=[min_val, max_val]
        ),
        yaxis=dict(
            title="Lab (%)",
            title_font=dict(size=18),
            tickfont=dict(size=14),
            range=[min_val, max_val]
        ),
        legend=dict(
            font=dict(size=14)
        ),
      width=1000,   # width in pixels
      height=600   # height in pixels

    )
    fig.show()

APP/PS1
r=0.328
Wilcoxon p=0.000233
APP/PS1-FIRE
r=-0.005
Wilcoxon p=1.78e-20
FIRE
r=0.028
Wilcoxon p=1.71e-08
WT
r=0.022
Wilcoxon p=1.34e-38


APP/PS1
r=0.041
Wilcoxon p=1.23e-09
APP/PS1-FIRE
r=0.171
Wilcoxon p=6.86e-16
FIRE
r=0.040
Wilcoxon p=1.56e-07
WT
r=0.170
Wilcoxon p=1.27e-33


APP/PS1
r=0.953
Wilcoxon p=4.85e-07
APP/PS1-FIRE
r=0.898
Wilcoxon p=0.000671
FIRE
r=0.920
Wilcoxon p=1
WT
r=0.756
Wilcoxon p=5.2e-19


APP/PS1
r=0.917
Wilcoxon p=2.47e-11
APP/PS1-FIRE
r=0.805
Wilcoxon p=0.27
FIRE
r=0.486
Wilcoxon p=0.0625
WT
r=0.777
Wilcoxon p=6.22e-11


In [ ]:

df = cell_df
for stain in df['stain'].unique():
  df_stain = df[(df['stain'] == stain) ]

  min_val = min(df_stain["model_pct"].quantile(0.025),
                df_stain["lab_pct"].quantile(0.025))
  max_val = max(df_stain["model_pct"].quantile(0.975),
                df_stain["lab_pct"].quantile(0.975))
  # Get correlation per line
  lines = df_stain['line'].unique()
  corr_dict = {}
  for line in lines:
      df_line = df_stain[df_stain['line'] == line]
      if len(df_line) > 1:  # Need at least 2 points for correlation
          corr, _ = pearsonr(df_line['model_pct'], df_line['lab_pct'])
          corr_dict[line] = corr
      else:
          corr_dict[line] = None

  fig = px.scatter(
      df_stain,
      x="model_pct",
      y="lab_pct",
      color="line",
      marginal_x="histogram",  # Histogram along X
      marginal_y="histogram",  # Histogram along Y
      opacity=0.6,
      template="plotly_white"
  )
  # Add 45° reference line
  fig.add_shape(
        type="line",
        x0=min_val,
        y0=min_val,
        x1=max_val,
        y1=max_val,
        line=dict(color="lightgray", width=1, dash="dash")  # lighter and thinner
  )

  fig.update_layout(
      title=f"{stain}: Model vs Lab Percentages with Distributions",
      xaxis=dict(title="Model (%)", range=[min_val, max_val]),
      yaxis=dict(title="Lab (%)", range=[min_val, max_val]),
      xaxis_title="Model (%)",
      yaxis_title="Lab (%)"
  )


  # Show correlation per line
  for line, corr in corr_dict.items():
      if corr is not None:
          print(f"Stain {stain}, Line {line}: Pearson r = {corr:.3f}")

  fig.show()

Stain SY38, Line WT: Pearson r = 0.099
Stain SY38, Line APP/PS1-FIRE: Pearson r = 0.090
Stain SY38, Line FIRE: Pearson r = 0.082
Stain SY38, Line APP/PS1: Pearson r = 0.578


Stain PSD, Line WT: Pearson r = 0.212
Stain PSD, Line APP/PS1-FIRE: Pearson r = 0.420
Stain PSD, Line FIRE: Pearson r = 0.582
Stain PSD, Line APP/PS1: Pearson r = 0.350


Stain MXO4, Line WT: Pearson r = 0.276
Stain MXO4, Line APP/PS1-FIRE: Pearson r = 0.944
Stain MXO4, Line FIRE: Pearson r = -0.025
Stain MXO4, Line APP/PS1: Pearson r = 0.980


Stain GFAP, Line WT: Pearson r = 0.834
Stain GFAP, Line APP/PS1-FIRE: Pearson r = 0.871
Stain GFAP, Line FIRE: Pearson r = 0.236
Stain GFAP, Line APP/PS1: Pearson r = 0.983


## Performance by Cell Lines

## SAM2 vs Cellpose

In [92]:
df1 = cell_df.copy()
df2 = sam_df.copy()

df1['dataset'] = 'Cellpose'
df2['dataset'] = 'SAM2'

combined = pd.concat([df1, df2], ignore_index=True)

combined['line'] = combined['line'].astype(str)
combined = combined[combined['lab_pct'] > 0.001]
fig = px.box(
    combined,
    x='stain',
    y='dice',
    color='dataset',       # different color for Dataset A / Dataset B
    points='all',          # show all points
    facet_col='line',      # one subplot per Line value
    category_orders={
        'line': sorted(combined['line'].unique()),
        'dataset': sorted(combined['dataset'].unique())
    },
    color_discrete_sequence=px.colors.qualitative.Alphabet # Using Set1 for more contrast
)

# Optional: make the layout scrollable if you have many Lines
fig.update_layout(
    height=500,
    width=400 * combined['line'].nunique(),  # scale width by number of facets
    title="Dice score comparison per Line and Stain",
    boxmode='group'
)

fig.show()

In [93]:

df1 = cell_df.copy()
df2 = sam_df.copy()

df1['dataset'] = 'Cellpose'
df2['dataset'] = 'SAM2'

combined = pd.concat([df1, df2], ignore_index=True)

combined['line'] = combined['line'].astype(str)
combined = combined[combined['lab_pct'] > 0.001]
fig = px.box(
    combined,
    x='stain',
    y='precision',
    color='dataset',       # different color for Dataset A / Dataset B
    points='all',          # show all points
    facet_col='line',      # one subplot per Line value
    category_orders={
        'line': sorted(combined['line'].unique()),
        'dataset': sorted(combined['dataset'].unique())
    },
    color_discrete_sequence=px.colors.qualitative.Alphabet
)

# Optional: make the layout scrollable if you have many Lines
fig.update_layout(
    height=500,
    width=400 * combined['line'].nunique(),  # scale width by number of facets
    title="Precision score comparison per Line and Stain",
    boxmode='group'
)

fig.show()

In [94]:


df1 = cell_df.copy()
df2 = sam_df.copy()

df1['dataset'] = 'Cellpose'
df2['dataset'] = 'SAM2'

combined = pd.concat([df1, df2], ignore_index=True)

combined['line'] = combined['line'].astype(str)
combined = combined[combined['lab_pct'] > 0.001]
fig = px.box(
    combined,
    x='stain',
    y='recall',
    color='dataset',       # different color for Dataset A / Dataset B
    points='all',          # show all points
    facet_col='line',      # one subplot per Line value
    category_orders={
        'line': sorted(combined['line'].unique()),
        'dataset': sorted(combined['dataset'].unique())
    },
    color_discrete_sequence=px.colors.qualitative.Alphabet
)

# Optional: make the layout scrollable if you have many Lines
fig.update_layout(
    height=500,
    width=400 * combined['line'].nunique(),  # scale width by number of facets
    title="Recall score comparison per Line and Stain",
    boxmode='group'
)

fig.show()

## Median Dice, Precision, Recall per file per run mode per model

In [ ]:
cell_sum_df

,stain,run_mode,file_name,dice,iou,precision,recall,model_pct,lab_pct,mouse,line
0,SY38,same_batch,PF390-S2-Stack-C-CROPPED-sy38,0.634914,0.465109,0.470793,0.974698,0.346769,0.167494,PF390,WT
1,SY38,same_batch,PF415-S1-Stack-A-Plaque-CROPPED-sy38,0.494815,0.328741,0.335165,0.944904,0.130995,0.046465,PF415,APP/PS1-FIRE
2,SY38,same_batch,PF390-S1-Stack-B-CROPPED-sy38,0.450114,0.290418,0.295521,0.943873,0.113167,0.035432,PF390,WT
3,SY38,same_batch,PF390-S1-Stack-C-CROPPED-sy38,0.488530,0.323215,0.327413,0.961841,0.234390,0.079787,PF390,WT
4,SY38,same_batch,PF415-S2-Stack-A-Plaque-CROPPED-sy38,0.537341,0.367373,0.371960,0.967524,0.321627,0.123648,PF415,APP/PS1-FIRE
...,...,...,...,...,...,...,...,...,...,...,...
11,GFAP,mixed_batch,PF386-S2-Stack-B-CROPPED-gfap,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PF386,WT
12,GFAP,mixed_batch,PF390-S2-Stack-C-CROPPED-gfap,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PF390,WT
13,GFAP,mixed_batch,PF390-S1-Stack-C-CROPPED-gfap,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PF390,WT
14,GFAP,mixed_batch,PF415-S2-Stack-C-CROPPED-gfap,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PF415,APP/PS1-FIRE


In [142]:
df1 = cell_sum_df.copy()
df2 = sam_sum_df.copy()

df1['dataset'] = 'Cellpose'
df2['dataset'] = 'SAM2'

combined = pd.concat([df1, df2], ignore_index=True)

combined['line'] = combined['line'].astype(str)
combined = combined[combined['lab_pct'] > 0.001]

lines_order = sorted(combined['line'].unique())
stains_order = sorted(combined['stain'].unique())
datasets_order = sorted(combined['dataset'].unique())

fig = px.box(
    combined,
    x='stain',
    y='dice',
    color='dataset',       # different color for Dataset A / Dataset B
    points='all',          # show all points
    facet_col='line',      # one subplot per Line value
    category_orders={
        'line': lines_order,
        'stain': stains_order,
        'dataset': datasets_order
    },
    color_discrete_sequence=['#8A2BE2', '#0000CD'] # Blue Violet (more saturated purple) and Medium Blue (more saturated blue)
)

# Calculate medians per stain, line, and dataset
medians = combined.groupby(['stain', 'line', 'dataset'])['dice'].median().reset_index()

# Add annotations for median values
# Determine the x-position for each box within its facet
# Each stain category on the x-axis within a facet is at integer positions (0, 1, 2, 3 for the four stains)
# Within each stain, boxes for different datasets are grouped.
# We need to calculate the offset for each dataset's box within the stain group.

# Spacing between the centers of the boxes within a group. This can be adjusted.
box_spacing = 0.2

for index, row in medians.iterrows():
    stain_idx = stains_order.index(row['stain'])
    line_idx = lines_order.index(row['line'])
    dataset_idx = datasets_order.index(row['dataset']) # Use the actual order of datasets

    # Calculate the offset from the center of the stain category
    # General formula for centering: offset = box_spacing * (dataset_idx - (n_datasets - 1) / 2)
    x_offset = box_spacing * (dataset_idx - (len(datasets_order) - 1) / 2)

    # The x-position for the annotation is the stain's integer position plus the calculated offset
    x_pos = stain_idx + x_offset

    fig.add_annotation(
        x=x_pos,
        y=row['dice'],
        text=f"{row['dice']:.3f}",
        showarrow=False,
        yshift=10, # Shift text up above the box
        # Reference to the correct facet's x-axis based on the line index (Plotly facets are 1-indexed)
        xref=f"x{line_idx+1}",
        font=dict(size=10, color='black', weight='bold'), # Make font bold
        xanchor='center', # Center the text horizontally
        yanchor='bottom' # Anchor the text below the y-coordinate
    )


# Optional: make the layout scrollable if you have many Lines
fig.update_layout(
    height=400,
    width=400 * combined['line'].nunique(),  # scale width by number of facets
    title="Dice score comparison per Line and Stain",
    boxmode='group'
)

fig.show()

In [143]:
df1 = cell_sum_df.copy()
df2 = sam_sum_df.copy()

df1['dataset'] = 'Cellpose'
df2['dataset'] = 'SAM2'

combined = pd.concat([df1, df2], ignore_index=True)

combined['line'] = combined['line'].astype(str)
combined = combined[combined['lab_pct'] > 0.001]

lines_order = sorted(combined['line'].unique())
stains_order = sorted(combined['stain'].unique())
datasets_order = sorted(combined['dataset'].unique())

fig = px.box(
    combined,
    x='stain',
    y='precision',
    color='dataset',       # different color for Dataset A / Dataset B
    points='all',          # show all points
    facet_col='line',      # one subplot per Line value
    category_orders={
        'line': lines_order,
        'stain': stains_order,
        'dataset': datasets_order
    },
    color_discrete_sequence=['#8A2BE2', '#0000CD'] # Blue Violet (more saturated purple) and Medium Blue (more saturated blue)
)

# Calculate medians per stain, line, and dataset
medians = combined.groupby(['stain', 'line', 'dataset'])['precision'].median().reset_index()

# Add annotations for median values
# Determine the x-position for each box within its facet
# Each stain category on the x-axis within a facet is at integer positions (0, 1, 2, 3 for the four stains)
# Within each stain, boxes for different datasets are grouped.
# We need to calculate the offset for each dataset's box within the stain group.

# Spacing between the centers of the boxes within a group. This can be adjusted.
box_spacing = 0.2

for index, row in medians.iterrows():
    stain_idx = stains_order.index(row['stain'])
    line_idx = lines_order.index(row['line'])
    dataset_idx = datasets_order.index(row['dataset']) # Use the actual order of datasets

    # Calculate the offset from the center of the stain category
    # General formula for centering: offset = box_spacing * (dataset_idx - (n_datasets - 1) / 2)
    x_offset = box_spacing * (dataset_idx - (len(datasets_order) - 1) / 2)

    # The x-position for the annotation is the stain's integer position plus the calculated offset
    x_pos = stain_idx + x_offset

    fig.add_annotation(
        x=x_pos,
        y=row['precision'],
        text=f"{row['precision']:.2f}", # Use .3f for precision
        showarrow=False,
        yshift=10, # Shift text up above the box
        # Reference to the correct facet's x-axis based on the line index (Plotly facets are 1-indexed)
        xref=f"x{line_idx+1}",
        font=dict(size=10, color='black', weight='bold'), # Make font bold
        xanchor='center', # Center the text horizontally
        yanchor='bottom' # Anchor the text below the y-coordinate
    )

# Optional: make the layout scrollable if you have many Lines
fig.update_layout(
    height=400,
    width=400 * combined['line'].nunique(),  # scale width by number of facets
    title="Precision score per Cell Line and Stain",
    boxmode='group'
)

fig.show()

In [144]:
df1 = cell_sum_df.copy()
df2 = sam_sum_df.copy()

df1['dataset'] = 'Cellpose'
df2['dataset'] = 'SAM2'

combined = pd.concat([df1, df2], ignore_index=True)

combined['line'] = combined['line'].astype(str)
combined = combined[combined['lab_pct'] > 0.001]

lines_order = sorted(combined['line'].unique())
stains_order = sorted(combined['stain'].unique())
datasets_order = sorted(combined['dataset'].unique())

fig = px.box(
    combined,
    x='stain',
    y='recall',
    color='dataset',       # different color for Dataset A / Dataset B
    points='all',          # show all points
    facet_col='line',      # one subplot per Line value
    category_orders={
        'line': lines_order,
        'stain': stains_order,
        'dataset': datasets_order
    },
    color_discrete_sequence=['#8A2BE2', '#0000CD'] # Blue Violet (more saturated purple) and Medium Blue (more saturated blue)
)

# Calculate medians per stain, line, and dataset
medians = combined.groupby(['stain', 'line', 'dataset'])['recall'].median().reset_index()

# Add annotations for median values
# Determine the x-position for each box within its facet
# Each stain category on the x-axis within a facet is at integer positions (0, 1, 2, 3 for the four stains)
# Within each stain, boxes for different datasets are grouped.
# We need to calculate the offset for each dataset's box within the stain group.

# Spacing between the centers of the boxes within a group. This can be adjusted.
box_spacing = 0.2

for index, row in medians.iterrows():
    stain_idx = stains_order.index(row['stain'])
    line_idx = lines_order.index(row['line'])
    dataset_idx = datasets_order.index(row['dataset']) # Use the actual order of datasets

    # Calculate the offset from the center of the stain category
    # General formula for centering: offset = box_spacing * (dataset_idx - (n_datasets - 1) / 2)
    x_offset = box_spacing * (dataset_idx - (len(datasets_order) - 1) / 2)

    # The x-position for the annotation is the stain's integer position plus the calculated offset
    x_pos = stain_idx + x_offset

    fig.add_annotation(
        x=x_pos,
        y=row['recall'],
        text=f"{row['recall']:.2f}", # Use .3f for recall
        showarrow=False,
        yshift=10, # Shift text up above the box
        # Reference to the correct facet's x-axis based on the line index (Plotly facets are 1-indexed)
        xref=f"x{line_idx+1}",
        font=dict(size=10, color='black', weight='bold'), # Make font bold
        xanchor='center', # Center the text horizontally
        yanchor='bottom' # Anchor the text below the y-coordinate
    )

# Optional: make the layout scrollable if you have many Lines
fig.update_layout(
    height=400,
    width=400 * combined['line'].nunique(),  # scale width by number of facets
    title="Recall score per Cell Line and Stain",
    boxmode='group'
)

fig.show()

In [ ]:

df = cell_sum_df
for stain in df['stain'].unique():
  df_stain = df[(df['stain'] == stain) ]
  df_stain = df_stain.sort_values(by = 'line')

  min_val = min(df_stain["model_pct"].quantile(0.025),
                df_stain["lab_pct"].quantile(0.025))
  max_val = max(df_stain["model_pct"].quantile(0.975),
                df_stain["lab_pct"].quantile(0.975))
  # Get correlation per line
  lines = df_stain['line'].unique()
  corr_dict = {}
  for line in lines:
      df_line = df_stain[df_stain['line'] == line]
      if len(df_line) > 1:  # Need at least 2 points for correlation
          corr, _ = pearsonr(df_line['model_pct'], df_line['lab_pct'])
          corr_dict[line] = corr
      else:
          corr_dict[line] = None

  fig = px.scatter(
      df_stain,
      x="model_pct",
      y="lab_pct",
      color="line",
      marginal_x="histogram",  # Histogram along X
      marginal_y="histogram",  # Histogram along Y
      opacity=0.6,
      template="plotly_white"
  )
  # Add 45° reference line
  fig.add_shape(
        type="line",
        x0=min_val,
        y0=min_val,
        x1=max_val,
        y1=max_val,
        line=dict(color="lightgray", width=1, dash="dash")  # lighter and thinner
  )

  fig.update_layout(
      title=f"{stain}: Model vs Lab Percentages with Distributions",
      xaxis=dict(title="Model (%)", range=[min_val, max_val]),
      yaxis=dict(title="Lab (%)", range=[min_val, max_val]),
      xaxis_title="Model (%)",
      yaxis_title="Lab (%)"
  )


  # Show correlation per line
  for line, corr in corr_dict.items():
      if corr is not None:
          print(f"Stain {stain}, Line {line}: Pearson r = {corr:.3f}")

  fig.show()

Stain SY38, Line APP/PS1: Pearson r = 0.968
Stain SY38, Line APP/PS1-FIRE: Pearson r = 0.904
Stain SY38, Line FIRE: Pearson r = 0.088
Stain SY38, Line WT: Pearson r = 0.931


Stain PSD, Line APP/PS1: Pearson r = 0.862
Stain PSD, Line APP/PS1-FIRE: Pearson r = 0.924
Stain PSD, Line FIRE: Pearson r = -0.434
Stain PSD, Line WT: Pearson r = 0.954


Stain MXO4, Line APP/PS1: Pearson r = 0.990
Stain MXO4, Line APP/PS1-FIRE: Pearson r = 0.946
Stain MXO4, Line FIRE: Pearson r = nan
Stain MXO4, Line WT: Pearson r = -0.091


/tmp/ipython-input-216219424.py:16: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.



Stain GFAP, Line APP/PS1: Pearson r = 0.997
Stain GFAP, Line APP/PS1-FIRE: Pearson r = 0.985
Stain GFAP, Line FIRE: Pearson r = nan
Stain GFAP, Line WT: Pearson r = 0.998


/tmp/ipython-input-216219424.py:16: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.



In [ ]:

df = sam_sum_df
for stain in df['stain'].unique():
  df_stain = df[(df['stain'] == stain) ]
  df_stain = df_stain.sort_values(by = 'line')

  min_val = min(df_stain["model_pct"].quantile(0.025),
                df_stain["lab_pct"].quantile(0.025))
  max_val = max(df_stain["model_pct"].quantile(0.975),
                df_stain["lab_pct"].quantile(0.975))
  # Get correlation per line
  lines = df_stain['line'].unique()
  corr_dict = {}
  for line in lines:
      df_line = df_stain[df_stain['line'] == line]
      if len(df_line) > 1:  # Need at least 2 points for correlation
          corr, _ = pearsonr(df_line['model_pct'], df_line['lab_pct'])
          corr_dict[line] = corr
      else:
          corr_dict[line] = None

  fig = px.scatter(
      df_stain,
      x="model_pct",
      y="lab_pct",
      color="line",
      marginal_x="histogram",  # Histogram along X
      marginal_y="histogram",  # Histogram along Y
      opacity=0.6,
      template="plotly_white"
  )
  # Add 45° reference line
  fig.add_shape(
        type="line",
        x0=min_val,
        y0=min_val,
        x1=max_val,
        y1=max_val,
        line=dict(color="lightgray", width=1, dash="dash")  # lighter and thinner
  )

  fig.update_layout(
      title=f"{stain}: Model vs Lab Percentages with Distributions",
      xaxis=dict(title="Model (%)", range=[min_val, max_val]),
      yaxis=dict(title="Lab (%)", range=[min_val, max_val]),
      xaxis_title="Model (%)",
      yaxis_title="Lab (%)"
  )


  # Show correlation per line
  for line, corr in corr_dict.items():
      if corr is not None:
          print(f"Stain {stain}, Line {line}: Pearson r = {corr:.3f}")

  fig.show()

Stain SY38, Line APP/PS1: Pearson r = 0.782
Stain SY38, Line APP/PS1-FIRE: Pearson r = 0.719
Stain SY38, Line FIRE: Pearson r = -0.992
Stain SY38, Line WT: Pearson r = 0.888


Stain PSD, Line APP/PS1: Pearson r = 0.464
Stain PSD, Line APP/PS1-FIRE: Pearson r = 0.677
Stain PSD, Line FIRE: Pearson r = 0.436
Stain PSD, Line WT: Pearson r = 0.887


Stain MXO4, Line APP/PS1: Pearson r = 0.994
Stain MXO4, Line APP/PS1-FIRE: Pearson r = 0.954
Stain MXO4, Line FIRE: Pearson r = nan
Stain MXO4, Line WT: Pearson r = 0.824


/tmp/ipython-input-1840995220.py:16: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.



Stain GFAP, Line APP/PS1: Pearson r = 0.956
Stain GFAP, Line APP/PS1-FIRE: Pearson r = 0.915
Stain GFAP, Line FIRE: Pearson r = 0.983
Stain GFAP, Line WT: Pearson r = 0.995


In [ ]:
# Define the order of run_modes to be consistent
df = sam_sum_df.copy()
df = df[df['lab_pct'] > 0.001]
run_modes = df['run_mode'].unique().tolist()
run_modes.sort()

# Create the boxplot (grouped by stain, colored by run_mode)
fig = px.box(
    df,
    x='stain',
    y='dice',
    color='run_mode',
    points='all',
    category_orders={'run_mode': run_modes},
    color_discrete_sequence=px.colors.qualitative.Set2,
)

# Calculate medians per stain and run_mode
medians = df.groupby(['stain', 'run_mode'])['dice'].median().reset_index()

# Number of run_modes (hues)
n_run_modes = len(run_modes)

# For each median, calculate approximate x position
# Plotly groups categories on x-axis as integers 0,1,... for each stain
# Boxes within group are spaced evenly around the integer
# We use a spacing factor (adjust if needed)

# Spacing factor for offset from the integer x position
offset = 0.2

# Create a mapping from stain to its integer position
stain_order = df['stain'].unique().tolist()
stain_pos = {stain: i for i, stain in enumerate(stain_order)}

for _, row in medians.iterrows():
    stain_idx = stain_pos[row['stain']]
    run_mode_idx = run_modes.index(row['run_mode'])

    # Calculate x position offset: centered around stain_idx
    # Spread run_modes evenly around stain_idx
    x_pos = stain_idx - offset*(n_run_modes-1)/2 + run_mode_idx*offset

    fig.add_annotation(
        x=x_pos,
        y=row['dice'],
        text=f"{row['dice']:.3f}",
        showarrow=False,
        yshift=15,
        font=dict(size=12, color='black', family='Arial Black, Arial, sans-serif'),  # bold font family
        align='center',
        xanchor='center',
    )

# Update layout for clarity
fig.update_layout(
    title="SAM2: Performance by Stain and Run Mode",
    yaxis_title="Dice Score",
    xaxis_title="Stain",
    boxmode='group',
    font=dict(size=14),
    legend_title_text="Run Mode",
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(l=50, r=50, t=60, b=50)
)

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()

## Overlap - summary results

In [ ]:
sam_sum_df

,stain,run_mode,file_name,dice,iou,precision,recall,model_pct,lab_pct,mouse,line
0,SY38,same_batch,PF390-S2-Stack-A-CROPPED-sy38,0.590076,0.418516,0.807674,0.464841,0.092356,0.160471,PF390,WT
1,SY38,same_batch,PF390-S1-Stack-C-CROPPED-sy38,0.347705,0.210438,0.742513,0.227003,0.024393,0.079787,PF390,WT
2,SY38,same_batch,PF415-S2-Stack-C-CROPPED-sy38,0.588174,0.416605,0.649132,0.537681,0.087171,0.105239,PF415,APP/PS1-FIRE
3,SY38,same_batch,PF390-S2-Stack-B-CROPPED-sy38,0.372194,0.228648,0.751044,0.247398,0.021081,0.063996,PF390,WT
4,SY38,same_batch,PF390-S2-Stack-C-CROPPED-sy38,0.524682,0.355640,0.807644,0.388551,0.080580,0.167494,PF390,WT
...,...,...,...,...,...,...,...,...,...,...,...
11,GFAP,mixed_batch,PF390-S2-Stack-C-CROPPED-gfap,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PF390,WT
12,GFAP,mixed_batch,PF390-S2-Stack-A-CROPPED-gfap,0.663569,0.496523,0.514409,0.934555,0.000887,0.000488,PF390,WT
13,GFAP,mixed_batch,PF386-S2-Stack-A-CROPPED-gfap,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PF386,WT
14,GFAP,mixed_batch,PF390-S1-Stack-B-CROPPED-gfap,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PF390,WT


In [ ]:
sam_sum_df.groupby(['stain', 'line']).agg({'dice':'median','iou':'median', 'precision':'median', 'recall':'median', 'model_pct':'median', 'lab_pct':'median'})

dice       iou  precision    recall  model_pct  \
stain line                                                               
GFAP  APP/PS1       0.688991  0.530765   0.886343  0.721399   0.004478   
      APP/PS1-FIRE  0.609552  0.438385   0.584983  0.735820   0.000971   
      FIRE          0.212766  0.135135   0.135135  0.500000   0.000012   
      WT            0.000000  0.000000   0.000000  0.000000   0.000000   
MXO4  APP/PS1       0.873814  0.776366   0.860403  0.850348   0.014187   
      APP/PS1-FIRE  0.747199  0.596423   0.808607  0.669492   0.001247   
      FIRE          0.000000  0.000000   0.000000  0.000000   0.000000   
      WT            0.313801  0.198866   0.199848  0.715599   0.000105   
PSD   APP/PS1       0.404434  0.253485   0.560199  0.318752   0.025932   
      APP/PS1-FIRE  0.431317  0.274955   0.572716  0.346435   0.015759   
      FIRE          0.438567  0.280875   0.550318  0.363873   0.036695   
      WT            0.423476  0.268651   0.594070  0.317305   0.021583   
SY38  APP/PS1       0.577239  0.406291   0.681355  0.513079   0.097101   
      APP/PS1-FIRE  0.522292  0.353447   0.748891  0.409766   0.042677   
      FIRE          0.554474  0.383671   0.738519  0.439763   0.084475   
      WT            0.518600  0.350075   0.746778  0.384922   0.038426   

                     lab_pct  
stain line                    
GFAP  APP/PS1       0.008965  
      APP/PS1-FIRE  0.000304  
      FIRE          0.000005  
      WT            0.000000  
MXO4  APP/PS1       0.013589  
      APP/PS1-FIRE  0.001206  
      FIRE          0.000000  
      WT            0.000042  
PSD   APP/PS1       0.045698  
      APP/PS1-FIRE  0.029599  
      FIRE          0.057073  
      WT            0.035734  
SY38  APP/PS1       0.125951  
      APP/PS1-FIRE  0.068347  
      FIRE          0.143471  
      WT            0.079787

In [ ]:
sam_sum_df[sam_sum_df['run_mode']=='diff_batch'].groupby(['stain', 'line']).agg({'dice':'median','iou':'median', 'precision':'median', 'recall':'median', 'model_pct':'median', 'lab_pct':'median'})

dice       iou  precision    recall  model_pct  \
stain line                                                               
GFAP  APP/PS1       0.688991  0.530765   0.886343  0.721399   0.004478   
      APP/PS1-FIRE  0.671501  0.508739   0.746815  0.561336   0.001805   
      FIRE          0.212766  0.135135   0.135135  0.500000   0.000012   
      WT            0.000000  0.000000   0.000000  0.000000   0.000000   
MXO4  APP/PS1       0.873814  0.776366   0.860403  0.850348   0.014187   
      APP/PS1-FIRE  0.448209  0.406140   0.407254  0.495104   0.007141   
      FIRE          0.000000  0.000000   0.000000  0.000000   0.000000   
      WT            0.307692  0.222222   0.303030  0.312500   0.000021   
PSD   APP/PS1       0.404434  0.253485   0.560199  0.318752   0.025932   
      APP/PS1-FIRE  0.431517  0.275220   0.500391  0.397754   0.033698   
      FIRE          0.438567  0.280875   0.550318  0.363873   0.036695   
      WT            0.483669  0.318977   0.524872  0.447766   0.026732   
SY38  APP/PS1       0.577239  0.406291   0.681355  0.513079   0.097101   
      APP/PS1-FIRE  0.451789  0.293064   0.742944  0.333511   0.045412   
      FIRE          0.554474  0.383671   0.738519  0.439763   0.084475   
      WT            0.502657  0.335834   0.682625  0.387595   0.043935   

                     lab_pct  
stain line                    
GFAP  APP/PS1       0.008965  
      APP/PS1-FIRE  0.002024  
      FIRE          0.000005  
      WT            0.000000  
MXO4  APP/PS1       0.013589  
      APP/PS1-FIRE  0.006029  
      FIRE          0.000000  
      WT            0.000020  
PSD   APP/PS1       0.045698  
      APP/PS1-FIRE  0.045458  
      FIRE          0.057073  
      WT            0.030962  
SY38  APP/PS1       0.125951  
      APP/PS1-FIRE  0.119831  
      FIRE          0.143471  
      WT            0.074480

In [ ]:
sam_df[(sam_df['run_mode']=='diff_batch') & (sam_df['lab_pct'] > 0)].groupby(['stain', 'line']).agg({'dice':'median','iou':'median', 'precision':'median', 'recall':'median', 'model_pct':'median', 'lab_pct':'median'})

dice       iou  precision    recall  model_pct  \
stain line                                                               
GFAP  APP/PS1       0.674786  0.509190   0.942197  0.719144   0.001890   
      APP/PS1-FIRE  0.617105  0.446241   0.751613  0.606876   0.000800   
      FIRE          0.542955  0.372642   0.372642  1.000000   0.000203   
      WT            0.261745  0.150579   0.150579  1.000000   0.000298   
MXO4  APP/PS1       0.857794  0.750997   0.816193  0.921537   0.009513   
      APP/PS1-FIRE  0.842691  0.728148   0.736394  0.986922   0.020354   
      FIRE          0.570129  0.398727   0.690476  0.401281   0.000211   
      WT            0.357309  0.217514   0.241135  0.666667   0.000146   
PSD   APP/PS1       0.503043  0.336044   0.636644  0.430681   0.015415   
      APP/PS1-FIRE  0.533601  0.363885   0.558720  0.520188   0.019710   
      FIRE          0.527135  0.357898   0.585316  0.475618   0.018828   
      WT            0.564591  0.393331   0.604496  0.556802   0.019626   
SY38  APP/PS1       0.561370  0.390212   0.674134  0.525578   0.031001   
      APP/PS1-FIRE  0.392304  0.244017   0.719921  0.271142   0.015067   
      FIRE          0.551778  0.381004   0.693300  0.456929   0.030064   
      WT            0.565160  0.393884   0.659524  0.513123   0.030197   

                     lab_pct  
stain line                    
GFAP  APP/PS1       0.003840  
      APP/PS1-FIRE  0.000863  
      FIRE          0.000060  
      WT            0.000046  
MXO4  APP/PS1       0.008628  
      APP/PS1-FIRE  0.015075  
      FIRE          0.000114  
      WT            0.000070  
PSD   APP/PS1       0.021267  
      APP/PS1-FIRE  0.021190  
      FIRE          0.021708  
      WT            0.021552  
SY38  APP/PS1       0.042580  
      APP/PS1-FIRE  0.042666  
      FIRE          0.044355  
      WT            0.043446

## What is proportion of images with at least some plaque in GFAP and MXO4?

In [ ]:
sam_df

,stain,run_mode,file_name,dice,iou,precision,recall,model_pct,lab_pct,mouse,line,file
0,SY38,same_batch,PF390-S1-Stack-B-CROPPED-sy38.tif_image_22,0.294690,0.172807,0.814901,0.179867,0.010825,0.049041,PF390,WT,PF390-S1-Stack-B-CROPPED-sy38
1,SY38,same_batch,PF390-S1-Stack-B-CROPPED-sy38.tif_image_23,0.485923,0.320937,0.771819,0.354580,0.023392,0.050917,PF390,WT,PF390-S1-Stack-B-CROPPED-sy38
2,SY38,same_batch,PF390-S1-Stack-B-CROPPED-sy38.tif_image_24,0.604773,0.433459,0.664812,0.554681,0.042124,0.050488,PF390,WT,PF390-S1-Stack-B-CROPPED-sy38
3,SY38,same_batch,PF390-S1-Stack-B-CROPPED-sy38.tif_image_25,0.636333,0.466633,0.589010,0.691924,0.056301,0.047927,PF390,WT,PF390-S1-Stack-B-CROPPED-sy38
4,SY38,same_batch,PF390-S1-Stack-B-CROPPED-sy38.tif_image_26,0.372854,0.229146,0.786722,0.244323,0.015713,0.050596,PF390,WT,PF390-S1-Stack-B-CROPPED-sy38
...,...,...,...,...,...,...,...,...,...,...,...,...
263,GFAP,mixed_batch,PF386-S2-Stack-B-CROPPED-gfap.tif_image_8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PF386,WT,PF386-S2-Stack-B-CROPPED-gfap
264,GFAP,mixed_batch,PF386-S2-Stack-B-CROPPED-gfap.tif_image_9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PF386,WT,PF386-S2-Stack-B-CROPPED-gfap
265,GFAP,mixed_batch,PF386-S2-Stack-B-CROPPED-gfap.tif_image_10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PF386,WT,PF386-S2-Stack-B-CROPPED-gfap
266,GFAP,mixed_batch,PF386-S2-Stack-B-CROPPED-gfap.tif_image_11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PF386,WT,PF386-S2-Stack-B-CROPPED-gfap


In [ ]:
# Calculate the count of images with lab_pct > 0 for each stain and line in diff_batch
positive_lab_pct_counts = sam_df[(sam_df['run_mode']=='diff_batch') & (sam_df['lab_pct'] > 0)].groupby(['stain', 'line']).count()['file'].reset_index()
positive_lab_pct_counts.columns = ['stain', 'line', 'positive_lab_pct_count']

# Calculate the total count of images, distinct mice, and list of files for each stain and line in diff_batch
total_counts = sam_df[(sam_df['run_mode']=='diff_batch')].groupby(['stain', 'line']).agg(
    total_count=('file', 'count'),
    distinct_stack=('file', 'nunique'),
    files=('file', lambda x: set(x)) # Add a column with a list of file names
).reset_index()


# Merge the two dataframes and calculate the proportion
proportion_df = pd.merge(positive_lab_pct_counts, total_counts, on=['stain', 'line'])
proportion_df['proportion_lab_pct_gt_0'] = proportion_df['positive_lab_pct_count'] / proportion_df['total_count']

display(proportion_df)

,stain,line,positive_lab_pct_count,total_count,distinct_stack,files,proportion_lab_pct_gt_0
0,GFAP,APP/PS1,101,101,4,"{PF336-S2-Stack-B-CROPPED-gfap, PF336-S1-Stack...",1.000000
1,GFAP,APP/PS1-FIRE,65,94,4,"{PF369-S1-Stack-A-Plaque-CROPPED-gfap, PF369-S...",0.691489
2,GFAP,FIRE,5,101,4,"{PF305-S1-Stack-A-CROPPED-gfap, PF305-S1-Stack...",0.049505
3,GFAP,WT,3,73,4,"{PF386-S2-Stack-A-CROPPED-gfap, PF386-S1-Stack...",0.041096
4,MXO4,APP/PS1,97,101,4,"{PF336-S2-Stack-A-Plaque-CROPPED-mxo4, PF336-S...",0.960396
5,MXO4,APP/PS1-FIRE,41,94,4,"{PF369-S1-Stack-A-Plaque-CROPPED-mxo4, PF369-S...",0.436170
6,MXO4,FIRE,5,101,4,"{PF305-S1-Stack-A-CROPPED-mxo4, PF305-S2-Stack...",0.049505
7,MXO4,WT,9,61,4,"{PF386-S1-Stack-A-CROPPED-mxo4, PF386-S1-Stack...",0.147541
8,PSD,APP/PS1,101,101,4,"{PF336-S1-Stack-A-Plaque-CROPPED-psd, PF336-S2...",1.000000
9,PSD,APP/PS1-FIRE,94,94,4,"{PF369-S2-Stack-B-CROPPED-psd, PF369-S1-Stack-...",1.000000


In [ ]:
## Plot GFAP and MXO4 distribution in lines where lab_pct > 0

In [ ]:
from scipy.stats import kruskal, wilcoxon
import scikit_posthocs as sp_post
import plotly.express as px
import pandas as pd

combined = cell_sum_df[(cell_sum_df['run_mode']=='diff_batch') & (cell_sum_df['lab_pct'] > 0)  & (cell_sum_df['stain'].isin(['GFAP', 'MXO4']))].copy()


# Melt the dataframe to have 'lab_pct' and 'model_pct' as a single column
combined_melted = combined.melt(
    id_vars=['stain', 'run_mode', 'file_name', 'dice', 'iou', 'precision', 'recall', 'mouse', 'line'],
    value_vars=['lab_pct', 'model_pct'],
    var_name='percentage_type',
    value_name='percentage_value'
)

fig = px.box(
    combined_melted,
    x='line',
    y='percentage_value',
    color='line',       # different color for Dataset A / Dataset B
    points='all',          # show all points
    facet_col='percentage_type',      # one subplot per Line value
    facet_row='stain',     # one row per Stain
    category_orders={
        'line': sorted(combined_melted['line'].unique()),
        'percentage_type': ['lab_pct', 'model_pct'] # Ensure order
    },
    color_discrete_sequence=px.colors.qualitative.Set2
)


fig.update_layout(
    height=800,
    width=400 * combined_melted['line'].nunique(),
    title="Lab vs Model Percentage Distribution by Stain, Line, and Model with Significance Tests",
    boxmode='group'
)

fig.show()

In [ ]:
from scipy.stats import kruskal, wilcoxon
import scikit_posthocs as sp_post
import plotly.express as px
import pandas as pd

combined = cell_df[(cell_df['run_mode']=='diff_batch') & (cell_df['lab_pct'] > 0)  & (cell_df['stain'].isin(['GFAP', 'MXO4']))].copy()


# Melt the dataframe to have 'lab_pct' and 'model_pct' as a single column
combined_melted = combined.melt(
    id_vars=['stain', 'run_mode', 'file_name', 'dice', 'iou', 'precision', 'recall', 'mouse', 'line', 'file'],
    value_vars=['lab_pct', 'model_pct'],
    var_name='percentage_type',
    value_name='percentage_value'
)

fig = px.box(
    combined_melted,
    x='line',
    y='percentage_value',
    color='line',       # different color for Dataset A / Dataset B
    points='all',          # show all points
    facet_col='percentage_type',      # one subplot per Line value
    facet_row='stain',     # one row per Stain
    category_orders={
        'line': sorted(combined_melted['line'].unique()),
        'percentage_type': ['lab_pct', 'model_pct'] # Ensure order
    },
    color_discrete_sequence=px.colors.qualitative.Set2
)


fig.update_layout(
    height=800,
    width=400 * combined_melted['line'].nunique(),
    title="Lab vs Model Percentage Distribution by Stain, Line, and Model with Significance Tests",
    boxmode='group'
)

fig.show()

In [134]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Filter both dataframes for relevant conditions (diff_batch, lab_pct > 0, GFAP & MXO4)
cell_filtered = cell_df[(cell_df['run_mode']=='diff_batch') & (cell_df['lab_pct'] > 0)  & (cell_df['stain'].isin(['GFAP', 'MXO4']))].copy()
sam_filtered = sam_df[(sam_df['run_mode']=='diff_batch') & (sam_df['lab_pct'] > 0)  & (sam_df['stain'].isin(['GFAP', 'MXO4']))].copy()

# Rename model_pct to indicate the source
cell_filtered = cell_filtered.rename(columns={'model_pct': 'Cellpose Model'})
sam_filtered = sam_filtered.rename(columns={'model_pct': 'SAM2 Model'})


# Select relevant columns and add a 'Lab' column based on lab_pct
lab_data = cell_filtered[['stain', 'line', 'lab_pct']].rename(columns={'lab_pct': 'Pixel Label %'})
lab_data['Source'] = 'Lab'

cellpose_model_data = cell_filtered[['stain', 'line', 'Cellpose Model']].rename(columns={'Cellpose Model': 'Pixel Label %'})
cellpose_model_data['Source'] = 'Cellpose Model'

sam_model_data = sam_filtered[['stain', 'line', 'SAM2 Model']].rename(columns={'SAM2 Model': 'Pixel Label %'})
sam_model_data['Source'] = 'SAM2 Model'

# Combine the dataframes
combined_percentage_data = pd.concat([lab_data, cellpose_model_data, sam_model_data], ignore_index=True)


# Create the boxplot grid
fig = px.box(
    combined_percentage_data,
    x='line', # Cell line on the x-axis within each facet
    y='Pixel Label %', # Percentage value on the y-axis
    color='Source', # Color boxes by percentage source (Lab, Cellpose Model, SAM Model)
    facet_row='stain', # Facet by stain (rows)
    # facet_col='Source', # Removed facet_col by Source
    points='all', # Show all points
    category_orders={
        'stain': sorted(combined_percentage_data['stain'].unique()),
        'Source': ['Lab', 'Cellpose Model', 'SAM2 Model'], # Ensure consistent color/grouping order
        'line': sorted(combined_percentage_data['line'].unique())
    },
    color_discrete_map={
        'Cellpose Model': '#8A2BE2',  # Blue Violet (kept the same)
        'SAM2 Model': '#4169E1',       # Royal Blue (lighter than Dark Blue, darker than Steel Blue)
        'Lab': '#333333'              # Dark Grey (kept the same)
    } # Apply custom color map
)


# Update layout for clarity
fig.update_layout(
    title="SAM2 and Cellpose comparison to Lab masks: Masked Pixel Label %",
    yaxis_title="Pixel Label %",
    boxmode='group' # Group boxes by Source within each Line on the x-axis
)

# Update facet titles (only for rows in this case)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1].strip()) if 'text' in a and isinstance(a['text'], str) and "=" in a['text'] else go.layout.Annotation())


fig.show()

In [ ]:
combined = sam_df[(sam_df['run_mode']=='diff_batch') & (sam_df['lab_pct'] > 0)  & (sam_df['stain'].isin(['GFAP', 'MXO4']))].copy()


# Melt the dataframe to have 'lab_pct' and 'model_pct' as a single column
combined_melted = combined.melt(
    id_vars=['stain', 'run_mode', 'file_name', 'dice', 'iou', 'precision', 'recall', 'mouse', 'line', 'file'],
    value_vars=['lab_pct', 'model_pct'],
    var_name='percentage_type',
    value_name='percentage_value'
)

fig = px.box(
    combined_melted,
    x='percentage_type',
    y='percentage_value',
    color='percentage_type',       # different color for Dataset A / Dataset B
    points='all',          # show all points
    facet_col='line',      # one subplot per Line value
    facet_row='stain',     # one row per Stain
    category_orders={
        'line': sorted(combined_melted['line'].unique()),
        'percentage_type': ['lab_pct', 'model_pct'] # Ensure order
    },
    color_discrete_sequence=px.colors.qualitative.Set2
)


fig.update_layout(
    height=800,
    width=400 * combined_melted['line'].nunique(),
    title="Lab vs Model Percentage Distribution by Stain, Line, and Model with Significance Tests",
    boxmode='group'
)

fig.show()

In [ ]:
from scipy.stats import kruskal
import scikit_posthocs as sp_post
import pandas as pd

# Use the 'cell_df' dataframe
# Filter for diff_batch, lab_pct > 0, and specific stains
cell_filtered = cell_df[(cell_df['run_mode']=='diff_batch') & (cell_df['lab_pct'] > 0)  & (cell_df['stain'].isin(['GFAP', 'MXO4']))].copy()

# Calculate the difference between lab_pct and model_pct
cell_filtered['pct_difference'] = cell_filtered['lab_pct'] - cell_filtered['model_pct']

print("Analyzing significance of the difference (lab_pct - model_pct) across lines for Cellpose dataset:")

kruskal_diff_results_cellpose = []
dunn_diff_results_cellpose = {}

# Perform Kruskal-Wallis test on the percentage difference across lines for each stain
for stain in cell_filtered['stain'].unique():
    subset = cell_filtered[(cell_filtered['stain'] == stain)].copy()

    if len(subset['line'].unique()) > 1:
        # Get percentage difference values grouped by line
        line_groups_diff = [subset[subset['line'] == line]['pct_difference'] for line in subset['line'].unique() if not subset[subset['line'] == line].empty] # Ensure groups are not empty

        if len(line_groups_diff) > 1 and all(len(group) > 0 for group in line_groups_diff): # Ensure at least 2 non-empty groups
           try:
               kw_stat, kw_p = kruskal(*line_groups_diff)
               kruskal_diff_results_cellpose.append({
                   'Stain': stain,
                   'Test': 'Kruskal-Wallis (Lab%-Model% across Lines)',
                   'Statistic': kw_stat,
                   'p_value': kw_p
               })

               # Post-hoc Dunn's test if Kruskal-Wallis is significant
               if kw_p < 0.05:
                   posthoc_dunn = sp_post.posthoc_dunn(subset, val_col='pct_difference', group_col='line', p_adjust='bonferroni')
                   dunn_diff_results_cellpose[stain] = posthoc_dunn

           except ValueError as e:
               print(f"Kruskal-Wallis test failed for Stain: {stain} - {e}")
        else:
           print(f"Not enough non-empty line groups for Kruskal-Wallis test for Stain: {stain}")
    else:
        print(f"Only one line available for Kruskal-Wallis test for Stain: {stain}")


kruskal_diff_df_cellpose = pd.DataFrame(kruskal_diff_results_cellpose)
print("\nKruskal-Wallis Test Results (Lab% - Model% across Lines) - Cellpose:")
display(kruskal_diff_df_cellpose)

print("\nDunn's Post-hoc Test Results (Lab% - Model% across Lines, Bonferroni corrected) - Cellpose:")
for stain, result_df in dunn_diff_results_cellpose.items():
    print(f"\nStain: {stain}")
    display(result_df)

Analyzing significance of the difference (lab_pct - model_pct) across lines for Cellpose dataset:

Kruskal-Wallis Test Results (Lab% - Model% across Lines) - Cellpose:


,Stain,Test,Statistic,p_value
0,MXO4,Kruskal-Wallis (Lab%-Model% across Lines),78.305603,7.086544e-17
1,GFAP,Kruskal-Wallis (Lab%-Model% across Lines),11.273026,1.033752e-02



Dunn's Post-hoc Test Results (Lab% - Model% across Lines, Bonferroni corrected) - Cellpose:

Stain: MXO4


,APP/PS1,APP/PS1-FIRE,FIRE,WT
APP/PS1,1.000000e+00,2.387998e-16,1.000000,1.000000
APP/PS1-FIRE,2.387998e-16,1.000000e+00,0.000193,0.000005
FIRE,1.000000e+00,1.930809e-04,1.000000,1.000000
WT,1.000000e+00,5.317442e-06,1.000000,1.000000



Stain: GFAP


,APP/PS1,APP/PS1-FIRE,FIRE,WT
APP/PS1,1.000000,1.000000,0.033842,0.299076
APP/PS1-FIRE,1.000000,1.000000,0.045535,0.347344
FIRE,0.033842,0.045535,1.000000,1.000000
WT,0.299076,0.347344,1.000000,1.000000


### One-Sided Independent Samples T-test (Comparing Lab% or Model% Between Two Lines)

In [ ]:
from scipy.stats import ttest_ind, shapiro, levene

# --- Parameters for the test ---
stain_to_test = 'GFAP' # Specify the stain
run_mode_to_test = 'diff_batch' # Specify the run mode
line1 = 'APP/PS1' # Specify the first line for comparison
line2 = 'APP/PS1-FIRE' # Specify the second line for comparison
percentage_type_to_compare = 'lab_pct' # Specify 'lab_pct' or 'model_pct'
alternative_hypothesis = 'greater' # Specify 'two-sided', 'less', or 'greater'


def t_test(df, stain_to_test, run_mode_to_test, line1, line2, percentage_type_to_compare, alternative_hypothesis):

  # --- Filter the data ---
  df_subset = df[(df['stain'] == stain_to_test) & (sam_df['run_mode'] == run_mode_to_test) & (sam_df['lab_pct'] > 0)].copy() # Filtering lab_pct > 0 as in previous analyses

  data_line1 = df_subset[df_subset['line'] == line1][percentage_type_to_compare].dropna()
  data_line2 = df_subset[df_subset['line'] == line2][percentage_type_to_compare].dropna()

  print(f"Performing one-sided t-test comparing {percentage_type_to_compare} between {line1} and {line2} for {stain_to_test} - {run_mode_to_test}")
  print(f"Alternative hypothesis: {percentage_type_to_compare} of {line1} is {alternative_hypothesis} than {percentage_type_to_compare} of {line2}")

  # --- Check assumptions (Normality and Equal Variances) ---
  alpha = 0.05

  # Check for Normality (Shapiro-Wilk test)
  shapiro_w1, shapiro_p1 = shapiro(data_line1)
  shapiro_w2, shapiro_p2 = shapiro(data_line2)
  print(f"\nNormality check for {line1} ({percentage_type_to_compare}): W={shapiro_w1:.3f}, p-value={shapiro_p1:.3g}")
  print(f"Normality check for {line2} ({percentage_type_to_compare}): W={shapiro_w2:.3f}, p-value={shapiro_p2:.3g}")

  if shapiro_p1 < alpha or shapiro_p2 < alpha:
      print("\nWarning: Normality assumption may be violated. Consider using a non-parametric test (e.g., Mann-Whitney U test).")

  # Check for Equal Variances (Levene's test)
  if len(data_line1) > 0 and len(data_line2) > 0:
      levene_stat, levene_p = levene(data_line1, data_line2)
      print(f"Equal variances check (Levene's test): Statistic={levene_stat:.3f}, p-value={levene_p:.3g}")

      if levene_p < alpha:
          print("Warning: Equal variances assumption may be violated. Consider using Welch's t-test (equal_var=False).")
          equal_variances = False
      else:
          equal_variances = True
  else:
      print("\nWarning: Not enough data points in one or both groups to perform Levene's test for equal variances.")
      equal_variances = True # Proceed with t-test assuming equal variances if Levene's cannot be performed


  # --- Perform the independent samples t-test ---
  if len(data_line1) >= 2 and len(data_line2) >= 2: # Need at least 2 data points in each group for t-test
      t_stat, p_value = ttest_ind(data_line1, data_line2, equal_var=equal_variances, alternative=alternative_hypothesis)

      print(f"\nIndependent Samples T-test Results:")
      print(f"  T-statistic: {t_stat:.3f}")
      print(f"  P-value ({alternative_hypothesis}): {p_value:.3g}")

      # Interpret the result
      if p_value < alpha:
          print(f"\nConclusion: There is a statistically significant difference in {percentage_type_to_compare} between {line1} and {line2} (p < {alpha}).")
          if alternative_hypothesis == 'greater':
              print(f"The {percentage_type_to_compare} of {line1} is significantly greater than the {percentage_type_to_compare} of {line2}.")
          elif alternative_hypothesis == 'less':
              print(f"The {percentage_type_to_compare} of {line1} is significantly less than the {percentage_type_to_compare} of {line2}.")
          else: # two-sided
              print(f"The {percentage_type_to_compare} of {line1} is significantly different from the {percentage_type_to_compare} of {line2}.")
      else:
          print(f"\nConclusion: There is no statistically significant difference in {percentage_type_to_compare} between {line1} and {line2} (p >= {alpha}).")

  else:
      print("\nNot enough data points in one or both groups to perform the t-test (need at least 2 in each).")

In [ ]:

# --- Parameters for the test ---
stain_to_test = 'GFAP' # Specify the stain
run_mode_to_test = 'diff_batch' # Specify the run mode
line1 = 'APP/PS1' # Specify the first line for comparison
line2 = 'APP/PS1-FIRE' # Specify the second line for comparison
percentage_type_to_compare = 'lab_pct' # Specify 'lab_pct' or 'model_pct'
alternative_hypothesis = 'greater' # Specify 'two-sided', 'less', or 'greater'
df = sam_df

t_test(df, stain_to_test, run_mode_to_test, line1, line2, percentage_type_to_compare, alternative_hypothesis)

Performing one-sided t-test comparing lab_pct between APP/PS1 and APP/PS1-FIRE for GFAP - diff_batch
Alternative hypothesis: lab_pct of APP/PS1 is greater than lab_pct of APP/PS1-FIRE

Normality check for APP/PS1 (lab_pct): W=0.877, p-value=1.16e-07
Normality check for APP/PS1-FIRE (lab_pct): W=0.792, p-value=3.55e-08

Equal variances check (Levene's test): Statistic=30.014, p-value=1.59e-07

Independent Samples T-test Results:
  T-statistic: 6.005
  P-value (greater): 6.18e-09

Conclusion: There is a statistically significant difference in lab_pct between APP/PS1 and APP/PS1-FIRE (p < 0.05).
The lab_pct of APP/PS1 is significantly greater than the lab_pct of APP/PS1-FIRE.


In [ ]:

# --- Parameters for the test ---
stain_to_test = 'MXO4' # Specify the stain
run_mode_to_test = 'diff_batch' # Specify the run mode
line1 = 'APP/PS1' # Specify the first line for comparison
line2 = 'APP/PS1-FIRE' # Specify the second line for comparison
percentage_type_to_compare = 'lab_pct' # Specify 'lab_pct' or 'model_pct'
alternative_hypothesis = 'less' # Specify 'two-sided', 'less', or 'greater'
df = sam_df

t_test(df, stain_to_test, run_mode_to_test, line1, line2, percentage_type_to_compare, alternative_hypothesis)

Performing one-sided t-test comparing lab_pct between APP/PS1 and APP/PS1-FIRE for MXO4 - diff_batch
Alternative hypothesis: lab_pct of APP/PS1 is less than lab_pct of APP/PS1-FIRE

Normality check for APP/PS1 (lab_pct): W=0.873, p-value=1.33e-07
Normality check for APP/PS1-FIRE (lab_pct): W=0.739, p-value=3.57e-07

Equal variances check (Levene's test): Statistic=1.876, p-value=0.173

Independent Samples T-test Results:
  T-statistic: -5.296
  P-value (less): 2.32e-07

Conclusion: There is a statistically significant difference in lab_pct between APP/PS1 and APP/PS1-FIRE (p < 0.05).
The lab_pct of APP/PS1 is significantly less than the lab_pct of APP/PS1-FIRE.


In [ ]:
from scipy.stats import mannwhitneyu

def mann_whitney_u_test(df, stain_to_test, run_mode_to_test, line1, line2, percentage_type_to_compare, alternative_hypothesis):

  # --- Filter the data ---
  df_subset = df[(df['stain'] == stain_to_test) & (df['run_mode'] == run_mode_to_test) & (df['lab_pct'] > 0)].copy() # Filtering lab_pct > 0 as in previous analyses

  data_line1 = df_subset[df_subset['line'] == line1][percentage_type_to_compare].dropna()
  data_line2 = df_subset[df_subset['line'] == line2][percentage_type_to_compare].dropna()

  print(f"Performing one-sided Mann-Whitney U test comparing {percentage_type_to_compare} between {line1} and {line2} for {stain_to_test} - {run_mode_to_test}")
  print(f"Alternative hypothesis: {percentage_type_to_compare} of {line1} is {alternative_hypothesis} than {percentage_type_to_compare} of {line2}")

  # --- Mann-Whitney U test (non-parametric) ---
  alpha = 0.05

  if len(data_line1) >= 2 and len(data_line2) >= 2: # Need at least 2 data points in each group
      try:
          u_stat, p_value = mannwhitneyu(data_line1, data_line2, alternative=alternative_hypothesis, method='auto')

          print(f"\nMann-Whitney U Test Results:")
          print(f"  U-statistic: {u_stat:.3f}")
          print(f"  P-value ({alternative_hypothesis}): {p_value:.3g}")

          # Interpret the result
          if p_value < alpha:
              print(f"\nConclusion: There is a statistically significant difference in the distribution of {percentage_type_to_compare} between {line1} and {line2} (p < {alpha}).")
              if alternative_hypothesis == 'greater':
                  print(f"The {percentage_type_to_compare} of {line1} is significantly greater than the {percentage_type_to_compare} of {line2}.")
              elif alternative_hypothesis == 'less':
                  print(f"The {percentage_type_to_compare} of {line1} is significantly less than the {percentage_type_to_compare} of {line2}.")
              else: # two-sided
                  print(f"The distribution of {percentage_type_to_compare} for {line1} is significantly different from that for {line2}.")
          else:
              print(f"\nConclusion: There is no statistically significant difference in the distribution of {percentage_type_to_compare} between {line1} and {line2} (p >= {alpha}).")

      except ValueError as e:
          print(f"\nCould not perform Mann-Whitney U test: {e}")

  else:
      print("\nNot enough data points in one or both groups to perform the Mann-Whitney U test (need at least 2 in each).")

In [ ]:

# --- Parameters for the test ---
stain_to_test = 'MXO4' # Specify the stain
run_mode_to_test = 'diff_batch' # Specify the run mode
line1 = 'APP/PS1' # Specify the first line for comparison
line2 = 'APP/PS1-FIRE' # Specify the second line for comparison
percentage_type_to_compare = 'lab_pct' # Specify 'lab_pct' or 'model_pct'
alternative_hypothesis = 'less' # Specify 'two-sided', 'less', or 'greater'
df = sam_df

mann_whitney_u_test(df, stain_to_test, run_mode_to_test, line1, line2, percentage_type_to_compare, alternative_hypothesis)

Performing one-sided Mann-Whitney U test comparing lab_pct between APP/PS1 and APP/PS1-FIRE for MXO4 - diff_batch
Alternative hypothesis: lab_pct of APP/PS1 is less than lab_pct of APP/PS1-FIRE

Mann-Whitney U Test Results:
  U-statistic: 929.000
  P-value (less): 4.03e-07

Conclusion: There is a statistically significant difference in the distribution of lab_pct between APP/PS1 and APP/PS1-FIRE (p < 0.05).
The lab_pct of APP/PS1 is significantly less than the lab_pct of APP/PS1-FIRE.


In [ ]:

# --- Parameters for the test ---
stain_to_test = 'MXO4' # Specify the stain
run_mode_to_test = 'diff_batch' # Specify the run mode
line1 = 'APP/PS1' # Specify the first line for comparison
line2 = 'APP/PS1-FIRE' # Specify the second line for comparison
percentage_type_to_compare = 'model_pct' # Specify 'lab_pct' or 'model_pct'
alternative_hypothesis = 'less' # Specify 'two-sided', 'less', or 'greater'
df = sam_df

mann_whitney_u_test(df, stain_to_test, run_mode_to_test, line1, line2, percentage_type_to_compare, alternative_hypothesis)

Performing one-sided Mann-Whitney U test comparing model_pct between APP/PS1 and APP/PS1-FIRE for MXO4 - diff_batch
Alternative hypothesis: model_pct of APP/PS1 is less than model_pct of APP/PS1-FIRE

Mann-Whitney U Test Results:
  U-statistic: 801.000
  P-value (less): 1.6e-08

Conclusion: There is a statistically significant difference in the distribution of model_pct between APP/PS1 and APP/PS1-FIRE (p < 0.05).
The model_pct of APP/PS1 is significantly less than the model_pct of APP/PS1-FIRE.


In [ ]:

# --- Parameters for the test ---
stain_to_test = 'GFAP' # Specify the stain
run_mode_to_test = 'diff_batch' # Specify the run mode
line1 = 'APP/PS1' # Specify the first line for comparison
line2 = 'APP/PS1-FIRE' # Specify the second line for comparison
percentage_type_to_compare = 'model_pct' # Specify 'lab_pct' or 'model_pct'
alternative_hypothesis = 'greater' # Specify 'two-sided', 'less', or 'greater'
df = sam_df

mann_whitney_u_test(df, stain_to_test, run_mode_to_test, line1, line2, percentage_type_to_compare, alternative_hypothesis)

Performing one-sided Mann-Whitney U test comparing model_pct between APP/PS1 and APP/PS1-FIRE for GFAP - diff_batch
Alternative hypothesis: model_pct of APP/PS1 is greater than model_pct of APP/PS1-FIRE

Mann-Whitney U Test Results:
  U-statistic: 4642.000
  P-value (greater): 3.46e-06

Conclusion: There is a statistically significant difference in the distribution of model_pct between APP/PS1 and APP/PS1-FIRE (p < 0.05).
The model_pct of APP/PS1 is significantly greater than the model_pct of APP/PS1-FIRE.


In [ ]:

# --- Parameters for the test ---
stain_to_test = 'GFAP' # Specify the stain
run_mode_to_test = 'diff_batch' # Specify the run mode
line1 = 'APP/PS1' # Specify the first line for comparison
line2 = 'APP/PS1-FIRE' # Specify the second line for comparison
percentage_type_to_compare = 'lab_pct' # Specify 'lab_pct' or 'model_pct'
alternative_hypothesis = 'greater' # Specify 'two-sided', 'less', or 'greater'
df = sam_df

mann_whitney_u_test(df, stain_to_test, run_mode_to_test, line1, line2, percentage_type_to_compare, alternative_hypothesis)

Performing one-sided Mann-Whitney U test comparing lab_pct between APP/PS1 and APP/PS1-FIRE for GFAP - diff_batch
Alternative hypothesis: lab_pct of APP/PS1 is greater than lab_pct of APP/PS1-FIRE

Mann-Whitney U Test Results:
  U-statistic: 4518.000
  P-value (greater): 2.2e-05

Conclusion: There is a statistically significant difference in the distribution of lab_pct between APP/PS1 and APP/PS1-FIRE (p < 0.05).
The lab_pct of APP/PS1 is significantly greater than the lab_pct of APP/PS1-FIRE.


In [ ]:

# --- Parameters for the test ---
stain_to_test = 'MXO4' # Specify the stain
run_mode_to_test = 'diff_batch' # Specify the run mode
line1 = 'APP/PS1' # Specify the first line for comparison
line2 = 'APP/PS1-FIRE' # Specify the second line for comparison
percentage_type_to_compare = 'lab_pct' # Specify 'lab_pct' or 'model_pct'
alternative_hypothesis = 'greater' # Specify 'two-sided', 'less', or 'greater'
df = cell_sum_df

mann_whitney_u_test(df, stain_to_test, run_mode_to_test, line1, line2, percentage_type_to_compare, alternative_hypothesis)

Performing one-sided Mann-Whitney U test comparing lab_pct between APP/PS1 and APP/PS1-FIRE for MXO4 - diff_batch
Alternative hypothesis: lab_pct of APP/PS1 is greater than lab_pct of APP/PS1-FIRE

Mann-Whitney U Test Results:
  U-statistic: 2.000
  P-value (greater): 0.867

Conclusion: There is no statistically significant difference in the distribution of lab_pct between APP/PS1 and APP/PS1-FIRE (p >= 0.05).


In [ ]:
from scipy.stats import kruskal, wilcoxon
import scikit_posthocs as sp_post
import plotly.express as px
import pandas as pd

df1 = cell_df[(cell_df['run_mode']=='diff_batch') & (cell_df['lab_pct'] > 0)  & (cell_df['stain'].isin(['GFAP', 'MXO4']))].copy()
df2 = sam_df[(sam_df['run_mode']=='diff_batch') & (sam_df['lab_pct'] > 0)  & (sam_df['stain'].isin(['GFAP', 'MXO4']))].copy()

df1['dataset'] = 'Cellpose'
df2['dataset'] = 'SAM2'

combined = pd.concat([df1, df2], ignore_index=True)

# Melt the dataframe to have 'lab_pct' and 'model_pct' as a single column
combined_melted = combined.melt(
    id_vars=['stain', 'run_mode', 'file_name', 'dice', 'iou', 'precision', 'recall', 'mouse', 'line', 'file', 'dataset'],
    value_vars=['lab_pct', 'model_pct'],
    var_name='percentage_type',
    value_name='percentage_value'
)

fig = px.box(
    combined_melted,
    x='percentage_type',
    y='percentage_value',
    color='dataset',       # different color for Dataset A / Dataset B
    points='all',          # show all points
    facet_col='line',      # one subplot per Line value
    facet_row='stain',     # one row per Stain
    category_orders={
        'line': sorted(combined_melted['line'].unique()),
        'dataset': sorted(combined_melted['dataset'].unique()),
        'percentage_type': ['lab_pct', 'model_pct'] # Ensure order
    },
    color_discrete_sequence=px.colors.qualitative.Set2
)

# Optional: make the layout scrollable if you have many Lines
fig.update_layout(
    height=800,
    width=400 * combined_melted['line'].nunique(),
    title="Lab vs Model Percentage Distribution by Stain, Line, and Model",
    boxmode='group',
    boxgroupgap=0.3, # Adjust gap between groups of boxes (lab_pct and model_pct)
    boxgap=0.1 # Adjust gap between boxes within a group (datasets)
)

fig.show()

# --- Statistical Tests and Results Description ---

print("\n--- Statistical Test Results ---")

# Wilcoxon test (lab_pct vs model_pct within each line/stain/dataset)
print("\nWilcoxon Signed-Rank Test Results (Lab vs Model % within Line/Stain/Dataset):")
for stain in combined_melted['stain'].unique():
    for line in combined_melted['line'].unique():
        for dataset in combined_melted['dataset'].unique():
            subset_melted = combined_melted[(combined_melted['stain'] == stain) & (combined_melted['line'] == line) & (combined_melted['dataset'] == dataset)].copy()

            lab_pct_subset = subset_melted[subset_melted['percentage_type'] == 'lab_pct']['percentage_value']
            model_pct_subset = subset_melted[subset_melted['percentage_type'] == 'model_pct']['percentage_value']

            if len(lab_pct_subset) >= 2 and len(model_pct_subset) >= 2 and len(lab_pct_subset) == len(model_pct_subset):
                try:
                    wilcoxon_stat, wilcoxon_p = wilcoxon(lab_pct_subset, model_pct_subset)
                    print(f"  - Stain: {stain}, Line: {line}, Dataset: {dataset}: Wilcoxon p={wilcoxon_p:.3g}")
                except (ValueError, TypeError) as e:
                     print(f"  - Stain: {stain}, Line: {line}, Dataset: {dataset}: Wilcoxon test failed - {e}")
            else:
                print(f"  - Stain: {stain}, Line: {line}, Dataset: {dataset}: Not enough paired data (need at least 2) for Wilcoxon test.")


# Kruskal-Wallis test (lab_pct across lines for each stain/run_mode/dataset)
print("\nKruskal-Wallis Test Results (Lab % across Lines for diff_batch run mode):")
for stain in combined['stain'].unique():
    # Since the combined data is filtered for diff_batch, we only need to iterate through dataset
    for dataset in combined['dataset'].unique():
        subset_original = combined[(combined['stain'] == stain) & (combined['run_mode'] == 'diff_batch') & (combined['dataset'] == dataset)].copy()

        if len(subset_original['line'].unique()) > 1:
             line_groups_lab_pct = [subset_original[subset_original['line'] == line]['lab_pct'] for line in subset_original['line'].unique() if not subset_original[subset_original['line'] == line].empty]

             if len(line_groups_lab_pct) > 1 and all(len(group) > 0 for group in line_groups_lab_pct):
                try:
                    kw_stat, kw_p = kruskal(*line_groups_lab_pct)
                    print(f"  - Stain: {stain}, Dataset: {dataset}: Kruskal-Wallis p={kw_p:.3g}")

                    # Post-hoc Dunn's test if Kruskal-Wallis is significant
                    if kw_p < 0.05:
                        print("    Dunn's Post-hoc Test Results (Bonferroni corrected):")
                        posthoc_dunn = sp_post.posthoc_dunn(subset_original, val_col='lab_pct', group_col='line', p_adjust='bonferroni')
                        display(posthoc_dunn)

                except ValueError as e:
                    print(f"  - Stain: {stain}, Dataset: {dataset}: Kruskal-Wallis test failed - {e}")

             else:
                print(f"  - Stain: {stain}, Dataset: {dataset}: Not enough non-empty line groups for Kruskal-Wallis test.")
        else:
            print(f"  - Stain: {stain}, Dataset: {dataset}: Only one line available for Kruskal-Wallis test.")


--- Statistical Test Results ---

Wilcoxon Signed-Rank Test Results (Lab vs Model % within Line/Stain/Dataset):
  - Stain: MXO4, Line: FIRE, Dataset: Cellpose: Wilcoxon p=0.0625
  - Stain: MXO4, Line: FIRE, Dataset: SAM2: Wilcoxon p=1
  - Stain: MXO4, Line: APP/PS1, Dataset: Cellpose: Wilcoxon p=0.0296
  - Stain: MXO4, Line: APP/PS1, Dataset: SAM2: Wilcoxon p=4.85e-07
  - Stain: MXO4, Line: APP/PS1-FIRE, Dataset: Cellpose: Wilcoxon p=9.09e-13
  - Stain: MXO4, Line: APP/PS1-FIRE, Dataset: SAM2: Wilcoxon p=9.09e-13
  - Stain: MXO4, Line: WT, Dataset: Cellpose: Wilcoxon p=0.0273
  - Stain: MXO4, Line: WT, Dataset: SAM2: Wilcoxon p=0.0391
  - Stain: GFAP, Line: FIRE, Dataset: Cellpose: Wilcoxon p=0.0625
  - Stain: GFAP, Line: FIRE, Dataset: SAM2: Wilcoxon p=0.0625
  - Stain: GFAP, Line: APP/PS1, Dataset: Cellpose: Wilcoxon p=4.1e-16
  - Stain: GFAP, Line: APP/PS1, Dataset: SAM2: Wilcoxon p=2.47e-11
  - Stain: GFAP, Line: APP/PS1-FIRE, Dataset: Cellpose: Wilcoxon p=3.04e-12
  - Stain: GFAP

,APP/PS1,APP/PS1-FIRE,FIRE,WT
APP/PS1,1.000000,4.208910e-05,0.012187,1.137493e-04
APP/PS1-FIRE,0.000042,1.000000e+00,0.000012,1.546272e-09
FIRE,0.012187,1.195443e-05,1.000000,1.000000e+00
WT,0.000114,1.546272e-09,1.000000,1.000000e+00


  - Stain: MXO4, Dataset: SAM2: Kruskal-Wallis p=2.04e-12
    Dunn's Post-hoc Test Results (Bonferroni corrected):


,APP/PS1,APP/PS1-FIRE,FIRE,WT
APP/PS1,1.000000,4.208910e-05,0.012187,1.137493e-04
APP/PS1-FIRE,0.000042,1.000000e+00,0.000012,1.546272e-09
FIRE,0.012187,1.195443e-05,1.000000,1.000000e+00
WT,0.000114,1.546272e-09,1.000000,1.000000e+00


  - Stain: GFAP, Dataset: Cellpose: Kruskal-Wallis p=7.21e-08
    Dunn's Post-hoc Test Results (Bonferroni corrected):


,APP/PS1,APP/PS1-FIRE,FIRE,WT
APP/PS1,1.000000,0.000604,0.000234,0.006177
APP/PS1-FIRE,0.000604,1.000000,0.038186,0.162942
FIRE,0.000234,0.038186,1.000000,1.000000
WT,0.006177,0.162942,1.000000,1.000000


  - Stain: GFAP, Dataset: SAM2: Kruskal-Wallis p=7.21e-08
    Dunn's Post-hoc Test Results (Bonferroni corrected):


,APP/PS1,APP/PS1-FIRE,FIRE,WT
APP/PS1,1.000000,0.000604,0.000234,0.006177
APP/PS1-FIRE,0.000604,1.000000,0.038186,0.162942
FIRE,0.000234,0.038186,1.000000,1.000000
WT,0.006177,0.162942,1.000000,1.000000
